# Lights, Camera, ReAction

This notebook builds a full sitcom script generation pipeline powered by ReAct-based agents, a framework that combines reasoning (thinking through a problem) with acting (taking structured steps).

We start by generating a sitcom concept from creative keywords, then outlining the pilot episode scene-by-scene. Scene 1 is generated directly from the outline to establish the world and tone. After that, each new scene is scripted, reviewed, and improved using specialized ReAct agents, a Character Agent, Comedy Agent, and Environment Agent, that simulate a real sitcom writers' room.

---

**Benefits of using ReAct agents:**

- **More structured and transparent thinking**: agents reason step-by-step before making edits.  
- **Dynamic adaptation**: agents flexibly plan the next creative moves based on scene history.  
- **Better long-term coherence**: scenes evolve logically, with tracked character growth, running jokes, emotional arcs, and worldbuilding.

---

After generation, each scene is summarized and stored as metadata, including characters, tone, location, and running jokes, in a vector database. This structured memory lets agents retrieve recent scene information to guide their evaluations and suggestions for new scenes.

By combining agent workflows and retrieval-based memory, the system produces scripts with greater consistency, character continuity, and narrative depth.


In [1]:
#!pip install datasets
#!pip install sentence-transformers faiss-cpu

## Mounting Drive and Appending System

In [2]:
# Import necessary modules
from google.colab import drive
import sys

# Mount Google Drive to access files stored in your Google Drive
drive.mount('/content/drive')

# NOTE: Update the paths below to match the location of your project files in Google Drive.
# Replace with your own directory if different.

# Add the main 'utils' directory to Python's module search path
# This allows you to import custom utility modules from this folder
sys.path.append("/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils")

# Add the 'agents' subdirectory to the module search path
# Useful if you have agent-specific Python scripts you want to import
sys.path.append("/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils/agents")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
'''
TO DELETE


import importlib
import agent_evaluation

importlib.reload(agent_evaluation)
'''

'\nTO DELETE\n\n\nimport importlib\nimport agent_evaluation\n\nimportlib.reload(agent_evaluation)\n'

## Importing Libraries and Packages

In [4]:
# Import sitcom agents responsible for generating specific scene elements
from agents.character_agent import CharacterAgent
from agents.comedy_agent import ComedicAgent
from agents.environment_agent import EnvironmentAgent
from agents.scene_planner_agent import ScenePlannerAgent

# Import evaluation functions to assess the quality of generated scenes
from agent_evaluation import (
    evaluate_character_agent_scene,
    evaluate_comedic_agent_scene,
    evaluate_environment_agent_scene
)

# Import text utility functions for formatting and extracting key components
from text_utils import (
    display_markdown_output,
    extract_scene,
    extract_title
)

# Import functions for generating sitcom concepts and outlines
from outline_generation import (
    generate_sitcom_pitch,
    generate_pilot_episode_outline
)

# Import functions to validate the structure and consistency of episodes
from script_review import (
    validate_episode_outline,
)

# Import scene writing functions
from screen_writing import (
    generate_scene_1_script,
    generate_scene
)

# Import vector database utilities for storing and retrieving scene information
from vector_db_utils import (
    summarize_scene,
    add_scene_to_vector_db,
    store_scene_in_vector_db
)

In [5]:
# Core Python Libraries
import os
import re

# Math and Data Manipulation
import numpy as np

# Display Utilities
from IPython.display import Markdown, display

# OpenAI API
import openai
from openai import OpenAI

# Google Colab Specific
from google.colab import userdata

# Sentence Embeddings and Vector Search
from sentence_transformers import SentenceTransformer
import faiss

### Package Versions

In [6]:
import pkg_resources

print("numpy:", np.__version__)
print("openai:", openai.__version__)
print("sentence-transformers:", pkg_resources.get_distribution("sentence-transformers").version)
print("faiss:", faiss.__version__ if hasattr(faiss, '__version__') else "version attribute not available")

numpy: 2.0.2
openai: 1.78.0
sentence-transformers: 4.1.0
faiss: 1.11.0



### 🔑 Initializing the OpenAI API Client

In [7]:
api_key = userdata.get('OPENAI_API_KEY')  # Securely fetch API key from Colab's stored credentials
client = OpenAI(api_key=api_key)          # Initialize the OpenAI client with the retrieved key

### Vector Database

This block initializes the sentence embedding model and sets up a FAISS index to store vector representations of scenes. The `all-MiniLM-L6-v2` model encodes scenes into fixed-size embeddings, and the FAISS index enables fast vector operations. The `scene_metadata` list stores contextual information linked to each scene embedding.


In [8]:
# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Vector store initialization
dimension = embedding_model.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(dimension)

# To track metadata
vector_metadata = []

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 🎬 Generating the Sitcom Pitch

This step generates a sitcom concept using the `generate_sitcom_pitch` function. The function takes in a dictionary of categorized keywords (e.g., setting, characters, themes, tone/genre) to guide the generation process.

- The language model plays the role of a **professional screenwriter**.
- It produces a 1-paragraph pitch that includes a **title** and a **detailed premise**, highlighting the setting, main characters, their dynamics, and the overall tone of the show.
- Keywords help ground the pitch in user-defined ideas, ensuring thematic and narrative consistency.

In [9]:
# Define key elements for sitcom generation: setting, characters, themes, and tone
keywords = {
    "setting": ["urban locksmith shop", "New York City"],
    "characters": ["ex-con protagonist", "estranged daughter", "quirky parole officer"],
    "themes": ["second chances", "odd couple dynamic"],
    "tone_genre": ["buddy comedy", "heartfelt absurdism"]
}

In [10]:
# Generate sitcom pitch using the language model
sitcom_pitch = generate_sitcom_pitch(client, keywords)

# Display the generated outline in formatted Markdown
display(Markdown(f"### Sitcom Concept:\n\n{sitcom_pitch}"))


### Sitcom Concept:

Title: "Second Lock"

"Second Lock" is a heartfelt yet absurd buddy comedy set in a small, quirky locksmith shop in the heart of New York City. The protagonist, Jimmy, is an ex-con turned locksmith who's trying to live a straight and narrow life, but his past keeps knocking. His estranged daughter, Sophie, a stubborn yet endearing teenager, unexpectedly shows up, challenging his new life and adding a layer of familial chaos. To add to the mix, Jimmy's eccentric parole officer, Earl, who has a bizarre obsession with locks and keys, frequently drops by, often blurring the line between professional supervision and genuine friendship. The show explores themes of second chances and the odd couple dynamic, with Jimmy and Earl's unlikely friendship serving as the hilarious backbone, while Sophie's attempts to reconnect with her father bring both comical and touching moments. The comedy is infused with heartfelt absurdism, as the characters navigate the challenges of life, family, and locksmithing in the Big Apple.

#### Extracting Title

Below, we extract the sitcom title from the generated pitch. This title will be reused by the script generation functions (both Baseline and ReAct versions).


In [11]:
# Extract title
sitcom_title = extract_title(sitcom_pitch)
print(sitcom_title)

Second Lock


### 📝 Generating a Pilot Episode Outline

The next part is to generate an episode outline for the pilot of the show, which is done using the `generate_pilot_episode_outline` function. This function takes in the sitcom pitch and produces a structured scene-by-scene breakdown of the pilot episode, simulating the voice of a **professional sitcom writer** preparing a network pitch.

- Begins with a concise **Episode Concept** (1–2 sentences).
- Breaks the episode into approximately 20 **numbered scenes**, each with a **title in quotation marks** and a brief description.
- Each scene summary includes what happens, who is involved, and the intended tone (e.g., funny, awkward, heartfelt).

This outline provides the narrative foundation for generating full scene scripts in the next stage.


In [12]:
# Generate episode outline from sitcom pitch using the language model
episode_outline = generate_pilot_episode_outline(client, sitcom_pitch)

# Display the generated outline in formatted Markdown
display(Markdown(f"### Sitcom Outline:\n\n{episode_outline}"))

### Sitcom Outline:

**Episode Concept:** Jimmy's estranged daughter, Sophie, unexpectedly shows up at his locksmith shop, the same day he receives a mysterious package from his past. Meanwhile, his parole officer, Earl, is dealing with his own issues, leading to a day full of chaos, laughter, and heart.

Scene 1: "Opening the Locks" Jimmy opens up his locksmith shop for the day, showcasing his quirky daily routine. The tone is light and funny.

Scene 2: "Unexpected Delivery" Jimmy receives a mysterious package, sparking curiosity and a bit of anxiety. The atmosphere is suspenseful yet comical.

Scene 3: "Earl's Obsession" Earl, the eccentric parole officer, visits Jimmy at the shop and shows off his latest lock-related obsession. The interaction is amusing and friendly.

Scene 4: "Blast from the Past" Jimmy opens the mysterious package to find a lock from his criminal past, causing him to reminisce. The tone is a mix of nostalgia and humor.

Scene 5: "Lock of Trouble" Jimmy and Earl try to open the lock, turning the situation into a humorous mess.

Scene 6: "Sophie's Arrival" Sophie, Jimmy's estranged daughter, unexpectedly shows up at the shop, creating an awkward and emotional moment.

Scene 7: "Father-Daughter Tensions" Sophie and Jimmy engage in a tense conversation about their past and why she has come. The scene is emotional yet comical due to their differences.

Scene 8: "Earl's Intervention" Earl tries to mediate the situation between Sophie and Jimmy, leading to a funny and awkward moment.

Scene 9: "A Locksmith's Lesson" Jimmy reluctantly agrees to teach Sophie some locksmithing skills, leading to a series of funny and touching moments.

Scene 10: "Earl's Dilemma" Earl receives a call about a personal issue, adding another layer of chaos to the day. The tone is comical and frantic.

Scene 11: "Locksmithing Mishaps" Sophie struggles with the locksmithing tasks, leading to hilarious blunders and bonding moments with Jimmy.

Scene 12: "Earl's Solution" Earl comes up with a humorous and unconventional solution to his problem. The scene is light-hearted and silly.

Scene 13: "Heart-to-Heart" Sophie and Jimmy have a heartfelt conversation about their relationship while working on locks. The tone is emotional and sincere.

Scene 14: "The Open Lock" Jimmy and Sophie manage to open the lock from the mysterious package, leading to a surprising revelation. The atmosphere is suspenseful and funny.

Scene 15: "Earl's Return" Earl returns to the shop, bringing his own chaos with him. The scene is chaotic and hilarious.

Scene 16: "Shared Histories" Jimmy, Sophie, and Earl share stories from their past, adding depth to their relationships. The tone is humorous and sentimental.

Scene 17: "Locking Up" As they close the shop, Jimmy, Sophie, and Earl reflect on the day's events. The atmosphere is light and reflective.

Scene 18: "Earl's Surprise" Earl reveals his personal issue has been resolved in an unexpected way. The scene is surprising and comical.

Scene 19: "A New Beginning" Sophie decides to stick around and learn more about locksmithing, signifying a new start for her and Jimmy. The tone is hopeful and heartwarming.

Scene 20: "Closing the Day" The day ends with Jimmy, Sophie, and Earl sharing a laugh, promising more hilarious adventures to come. The atmosphere is light and funny.

#### Exporting Outline and Title Name

Below, we export the sitcom title and episode outline. This ensures that both the Baseline and ReAct models use the same input for consistency in script generation.

In [13]:
# Define file paths for saving the sitcom title and episode outline
title_path = "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/pilot episode data/pilot episode 3/sitcom_title_3.txt"
outline_path = "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/pilot episode data/pilot episode 3/episode_outline_3.txt"

# Write the sitcom title to a text file
with open(title_path, "w") as f:
    f.write(sitcom_title)

# Write the episode outline to a text file
with open(outline_path, "w") as f:
    f.write(episode_outline)


### ✅ Episode Validation

To ensure that the sitcom pilot outline is coherent and aligns with the original pitch, we use the `validate_episode_outline` function. This function simulates the voice of a **veteran sitcom writer** who evaluates whether the episode outline makes narrative sense and fits the tone and premise.

- Checks for overall **coherence**, including consistency of tone, logical character progression, and structural viability.
- Returns a clear **"Yes" or "No"** on coherence, along with 2–4 bullet points of reasoning.
- If the episode is not coherent, it also provides concrete suggestions for improvement.

This validation step helps maintain narrative quality before moving into full scene scripting.


**TODO**

- Make sure it can re-generate the script
- Delte timedelta

In [14]:
# Validate the episode outline for coherence and consistency using the language model
validation = validate_episode_outline(client, sitcom_pitch, episode_outline)

# Display the validation result in formatted Markdown
display(Markdown(f"### Coherence Evaluation:\n\n{validation}"))

### Coherence Evaluation:

Coherence: Yes

Reasoning:
- The outline follows the premise of the sitcom pitch very well. The characters are consistent with their descriptions, and the plot progresses logically from Jimmy's regular day at the shop, to the arrival of Sophie and her decision to stay.
- The tone remains consistent throughout the episode, balancing humor with heartfelt moments, which aligns with the show's description of being a "heartfelt yet absurd buddy comedy."
- The episode outline introduces and explores the dynamics between the three main characters, setting the stage for future episodes. It also incorporates the locksmithing theme throughout, which is a unique aspect of the show.
- The subplot of Earl dealing with a personal issue adds another layer of humor and chaos to the episode, which further contributes to the overall tone of the show.

## Scene 1 Generation

In [15]:
# Extract the description for Scene 1 from the episode outline
scene_1_desc = extract_scene(episode_outline, 1)
print(scene_1_desc)

Scene 1: "Opening the Locks" Jimmy opens up his locksmith shop for the day, showcasing his quirky daily routine. The tone is light and funny.


In [16]:
# Generate the full script for Scene 1 based on the scene description
scene_1_script = generate_scene_1_script(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_1_desc,
    scene_index=0,
    rag_context=None,
)

print(scene_1_script)

# Scene 1

INT. JIMMY'S LOCKSMITH SHOP - MORNING

JIMMY, mid 40s, opens the door to his shop, a jumble of keys jingling from his belt. He flips the sign from CLOSED to OPEN. He's wearing a t-shirt that says, "I've Got the Key to Your Heart."

JIMMY
(To himself)
Another day, another lock to pick. 

Jimmy begins his peculiar morning routine. He pulls out a lock from his pocket and tries to pick it using a hairpin.

JIMMY
(To the lock)
You're a tough one, aren't you? 

[LAUGH TRACK]

Suddenly, a cat, WHISKERS, jumps on the counter. Jimmy reacts with surprise.

JIMMY
Whiskers! You sneaky little locksmith, you!

[LAUGH TRACK]

Jimmy then pulls out a fancy looking key from a drawer.

JIMMY
(To the key)
Ah, the master key. Opens all doors but can't open a can of tuna.

[LAUGH TRACK]

Jimmy attempts to use the master key to open a can of tuna for Whiskers. It doesn't work. He sighs, then pulls out a can opener from another drawer and opens the can.

JIMMY
(To Whiskers)
Fine, you win this round

### 🗂️ Adding Metadata to the Vector Database

To extract structured information from each generated sitcom scene, we use th `summarize_scene` function. This function simulates the head writer of the show and analyzes the scene script to produce a concise summary along with key metadata.

- **Summary**: A 100–150 word overview of the scene’s key actions, character dynamics, and notable moments.
- **Characters**: A list of characters who appear or speak in the scene.
- **Location**: The main setting of the scene, if clearly identifiable (e.g., “locksmith shop,” “apartment hallway”).
- **Recurring Joke**: Any running gag or callback featured in the scene.
- **Emotional Tone**: A 1–2 word description of the scene’s mood (e.g., “chaotic,” “hopeful,” “awkward”).

The metadata is returned as a dictionary and appended to the `scene_metadata` list using the `add_scene_to_vector_db` function. This step is crucial: instead of supplying the full script to the next generation phase, we pass a concise summary and key elements to guide smoother, context-aware scene generation.


In [17]:
# Store the generated Scene 1 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_1_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 1 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted


## 🎬 Scene 2 Generation

The next part is to generate Scene 2. This is one of the most important scenes in the script, as it’s where the ReAct (Reasoning + Acting) system comes into play.

Metadata from Scene 1 is retrieved and interpreted by the `CharacterAgent`, `ComedyAgent`, and `EnvironmentAgent`. Each agent reasons over the metadata independently and provides creative suggestions based on its area of expertise—such as character consistency, comedic dynamics, or environmental context.

These agent responses are then passed to the `ScenePlannerAgent`, which integrates their inputs and plans the next scene accordingly. This collaborative reasoning-and-acting process ensures that each scene builds logically and creatively on what came before.

The advantage of using ReAct-style agents over purely role-based agents is that each agent not only plays a role but also actively reasons and adapts based on prior context—making the generation more dynamic, context-aware, and narratively coherent.


In [18]:
# Extract the description for Scene 2 from the episode outline
scene_2_desc = extract_scene(episode_outline, 2)
print(scene_2_desc)

Scene 2: "Unexpected Delivery" Jimmy receives a mysterious package, sparking curiosity and a bit of anxiety. The atmosphere is suspenseful yet comical.


### 🎭 Character Agent

The first step is to analyze the characters using the `CharacterAgent`. This agent is responsible for tracking which characters appear in each scene, evaluating their consistency with past behavior, and recommending meaningful interactions to advance the story.

The `CharacterAgent` operates using a **ReAct workflow**, where each phase reflects a specific role in a sitcom writers’ room:

- **Think**: Identifies which characters are present in the current scene and determines the relevant prior scenes to draw context from — like a **Writers' Assistant** reviewing past episodes. This step also retrieves the relevant script metadata used to guide character evaluation.
- **Act**: Retrieves character histories by analyzing summaries from earlier scenes — like a **Script Coordinator** maintaining continuity.
- **Observe**: Checks whether characters behave consistently with their established traits and arcs — like a **Head Writer** reviewing for narrative logic.
- **Recommend**: Suggests specific character interactions that align with the scene’s tone or resolve inconsistencies — like a **Co-Executive Producer** steering character development.

This structure ensures character arcs are coherent, grounded, and creatively engaging throughout the scriptwriting process.


In [19]:
'''
import importlib
import agents.character_agent
importlib.reload(agents.character_agent)
from agents.character_agent import CharacterAgent
'''

'\nimport importlib\nimport agents.character_agent\nimportlib.reload(agents.character_agent)\nfrom agents.character_agent import CharacterAgent\n'

In [20]:
# Initialize the character agent with the language model client, metadata store, and total number of scenes
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=1
)

# Evaluate Scene 2 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_2_desc,
    scene_number=2
)

🔁 Agent will consider the last 1 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1]
Scene 2 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: The planned scene plays well with Jimmy's established personality traits. His quirky nature could manifest in his reaction to the mysterious package, while his resilience could see him handle the situation logically. His good-humored nature might add a comical spin to the suspenseful atmosphere. The scene doesn't contradict any previously established traits or relationships.

Interaction Improvement Recommendations:
1. Jimmy opens the mysterious package with a hairpin rather than a conventional tool like a knife or scissors — (This fits because, in Scene 1, Jimmy is portrayed as a quirky individual who uses unconventional methods to perform everyday tasks. Using a hairpin to open a package not only highlights his unique personality but also ties back to his unusual morning routine of trying to pick a lock with

### 🎤 Comedic Agent


The `ComedicAgent` is responsible for maintaining the tonal consistency of humor throughout the sitcom and making targeted punch-up recommendations to enhance comedic value.

This agent operates using a **ReAct workflow**, where each phase mimics a specific role in a sitcom writers’ room and draws on **prior scene metadata** for context:

- **Think**: Logs which scene is under review and determines which previous scenes will provide comedic context — like a **Writers' Assistant** flagging what material to reference.
- **Act**: Retrieves summaries and running jokes from recent scenes by referencing stored **vector metadata** — like a **Script Coordinator** tracking comedic beats.
- **Observe**: Evaluates whether the new scene's humor is consistent with previous tone and joke usage — like a **Head Writer** preserving the show's comedic style.
- **Recommend**: Suggests two realistic comedic improvements grounded in the scene’s context and character behavior — like a **Co-Executive Producer** overseeing punch-up work.

This design ensures each scene contributes to a cohesive comedic voice and builds on established humor organically, enhancing audience engagement.


**Maybe better note how this suggestions also based off inconsistency or not**

In [21]:
'''
import importlib
import agents.comedy_agent
importlib.reload(agents.comedy_agent)
from agents.comedy_agent import ComedicAgent
'''

'\nimport importlib\nimport agents.comedy_agent\nimportlib.reload(agents.comedy_agent)\nfrom agents.comedy_agent import ComedicAgent\n'

In [22]:
# Initialize the comedy agent with the language model client, metadata store, and scene history window
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=1
)

# Evaluate Scene 2 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_2_desc,
    scene_number=2
)

🔁 Agent will consider the last 1 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1]
Scene 2 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The suspenseful humor in the new scene is consistent with the quirky and playful humor from the previous scene. Jimmy's character is maintained as he faces unexpected situations with a sense of humor.

Comedic Improvement Recommendations:
1. [Suggestion] — As Jimmy tries to guess what's inside the mysterious package, his roommate, Martin, walks in. Jimmy holds up the package and asks, "Guess what's inside?" Martin, who is always diet-conscious and obsessed with health foods, responds without missing a beat, "Oh, it's my organic, gluten-free, non-GMO, cruelty-free, locally sourced, vegan protein powder?" Jimmy shakes the box next to his ear, "Doesn't sound like it." — (This suggestion reinforces Martin's character quirk of being health conscious and adds a light-hearted joke that fits within the context 

### 🌆 Environment Agent

The `EnvironmentAgent` is responsible for ensuring spatial continuity and logical scene transitions in a sitcom setting. It verifies that each new scene's location aligns naturally with prior ones and suggests immersive environmental details to enhance tone and narrative flow.

This agent operates using a **ReAct workflow**, where each step mimics a professional role in a sitcom writers' room and draws on **prior scene metadata**:

- **Think**: Logs which scene is being reviewed and determines which previous environments will provide context — like a **Writers' Assistant** organizing location continuity.
- **Act**: Analyzes the current scene’s setting and identifies key environmental elements — like a **Script Coordinator** extracting staging needs from a scene.
- **Observe**: Evaluates whether the environment transition feels natural given recent scenes — like a **Head Writer** safeguarding spatial logic and pacing.
- **Recommend**: Suggests sensory or scenic details to support the environment or smooth jarring transitions — like a **Co-Executive Producer** fine-tuning atmosphere with practical details.

Together, these steps ensure that every scene change feels intentional, immersive, and narratively cohesive — supporting both visual consistency and comedic timing.



In [23]:
'''
# TO DELETE
import importlib
import agents.environment_agent
importlib.reload(agents.environment_agent)
from agents.environment_agent import EnvironmentAgent
'''

'\n# TO DELETE\nimport importlib\nimport agents.environment_agent\nimportlib.reload(agents.environment_agent)\nfrom agents.environment_agent import EnvironmentAgent\n'

In [24]:
# Initialize the environment agent with the language model client, metadata store, and scene history window
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,  # prior scene metadata
    num_scenes=1
)

# Evaluate Scene 2 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_2_desc,
    scene_number=2
)

🔁 Environment Agent will consider the last 1 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1]
Scene 2 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: It is logical for a character to move from their place of work to their home, in this case, from Jimmy's Locksmith Shop to Jimmy's Living Room. This is a common transition in many sitcoms and does not require any additional setup.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As Jimmy enters the living room, he trips over a large package that's mysteriously placed near the entrance. The package is wrapped in brown paper and tied with a thick twine, looking out of place in Jimmy's otherwise neat and organized living room. This could add a touch of humor and intrigue.
- [Suggestion 2]: Jimmy's pet parrot squawks loudly from its cage as Jimmy walks in, reacting to the mysterious package in the room. The parrot keeps repeating the phrase "Stranger Danger!" whic

### 🧠 Scene Planner Agent

The `ScenePlannerAgent` acts as the **Executive Producer**, responsible for synthesizing suggestions from all other agents and outlining a coherent, engaging plan for the next scene.

Rather than simply selecting from character, comedic, and environment recommendations, this agent **distills key themes and feedback** into a unified set of creative objectives that drive the sitcom forward.

The agent uses a **ReAct-based synthesis approach**, interpreting prior agent outputs and generating new, structured goals:

- **Character Goals (2)** — derived from the `CharacterAgent`'s recommendations, focusing on growth, conflict, or relationships.
- **Comedic Goal (1)** — distilled from tone analysis and punch-up suggestions made by the `ComedicAgent`.
- **Environment Detail (1)** — extracted from the `EnvironmentAgent`'s observations and setting enhancements.
- **Creative Suggestion (1)** — an original narrative nudge that naturally integrates all the above goals into a sitcom-appropriate next step.

This agent ensures every new scene is **purposeful, tonally consistent, and narratively aligned**, helping maintain momentum and coherence throughout the episode.


In [25]:
'''

# TO DELETE
import importlib
import agents.scene_planner_agent
importlib.reload(agents.scene_planner_agent)
from agents.scene_planner_agent import ScenePlannerAgent

'''

'\n\n# TO DELETE\nimport importlib\nimport agents.scene_planner_agent\nimportlib.reload(agents.scene_planner_agent)\nfrom agents.scene_planner_agent import ScenePlannerAgent\n\n'

In [26]:
# Initialize the scene planner agent with the language model client
scene_planner_agent = ScenePlannerAgent(client=client)

# Generate a plan for Scene 2 by integrating recommendations from all three agents
scene_2_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=2
)

print(scene_2_plan)

Scene Plan:
Character Goals:
- [Goal 1]: Establish Jimmy's unconventional approach to solving problems by using a hairpin to open the mysterious package, reinforcing his quirky personality.
- [Goal 2]: Demonstrate Jimmy's endearing relationship with his pets by engaging in a humorous dialogue with his cat Whiskers and parrot, increasing the comedic value and adding depth to his character.

Comedic Goal:
- [Goal]: Incorporate a humor-filled interaction involving Jimmy's health-conscious roommate, Martin, and their over-dramatic neighbor, Mrs. Jenkins, around the suspense of the mysterious package. The comedic exchanges should maintain the light-hearted atmosphere and highlight the contrast between the characters' reactions.

Environment Detail:
- [Detail]: The living room should be impeccably organized, except for the unusual presence of the mysterious package by the entrance. Jimmy's parrot, from its cage, plays a comedic role by repeating phrases that Jimmy has taught it, adding to th

### 🎬 Generating Scene 2 Script

In [27]:
'''
import importlib
import screen_writing
importlib.reload(screen_writing)
'''

'\nimport importlib\nimport screen_writing\nimportlib.reload(screen_writing)\n'

In [28]:
# Generate the full script for Scene 2 based on the structured scene plan
scene_2_script = generate_scene(
    client=client,
    scene_plan=scene_2_plan,
    scene_number= 2
)

print(scene_2_script)

INT. LIVING ROOM - DAY

JIMMY, a man of unconventional habits, is on the floor, a MYSTERIOUS PACKAGE beside him. He holds a HAIRPIN in his hand.

JIMMY
(To Whiskers, the cat)
Whiskers, my dear, when conventional methods fail, it's time to think out of the box.

[Laugh Track]

Jimmy's PARROT, from its cage, squawks.

PARROT
Out of the box, out of the box!

[Laugh Track]

As Jimmy leans in to open the package, Whiskers, the cat, knocks over a PLANT. Jimmy jumps back.

JIMMY
(To Whiskers)
Well, thanks for the distraction, Whiskers!

[Laugh Track]

The front door opens, and MARTIN, Jimmy's health-conscious roommate, walks in, holding a bag of organic groceries.

MARTIN
(Sees plant on floor)
Jimmy, did you try to feed the plant your junk food again?

[Laugh Track]

JIMMY
(Grins)
Not this time, Martin. Just a minor cat-astrophe.

[Laugh Track]

Suddenly, the door bursts open, and MRS. JENKINS, their over-dramatic neighbor, rushes in.

MRS. JENKINS
(Breathless)
I saw it from my window... The 

### Adding Scene 2 to Vector Database

In [29]:
# Store the generated Scene 2 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_2_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 2 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. Hi

## 🎬 Scene 3 Generation

Scene 3 (through 20) will follow the same exact structure as Scene 2

In [30]:
# Extract the description for Scene 3 from the episode outline
scene_3_desc = extract_scene(episode_outline, 3)
print(scene_3_desc)

Scene 3: "Earl's Obsession" Earl, the eccentric parole officer, visits Jimmy at the shop and shows off his latest lock-related obsession. The interaction is amusing and friendly.


### Character Agent

In [31]:
# Re-initialize the character agent with the language model client, metadata store, and total number of scenes
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=2
)

# Evaluate Scene 3 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_3_desc,
    scene_number=3
)

🔁 Agent will consider the last 2 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1, 2]
Scene 3 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Earl's eccentricity is consistent with his previous obsession with locks, and his friendly visit to Jimmy aligns with their established relationship. Jimmy's resilience, good humor, and unconventional habits are maintained as he interacts with Earl. The scene's humor aligns with the sitcom's tone, and no contradictions are present based on the past scenes.

Interaction Improvement Recommendations:
1. Earl, fascinated by Jimmy's hairpin, asks if he can borrow it to examine its potential as a lock-picking tool. Jimmy, amused, hands it over while jokingly warning Earl not to lose it. — (This interaction showcases Earl's eccentric obsession with locks from Scene 3 and Jimmy's unconventional obsession with hairpins from Scenes 1 and 2.)
2. As they continue their chat, Earl, in his sociable manner, asks Jimmy abo

### Comedy Agent

In [32]:
# Re-initialize the comedy agent with the language model client, metadata store, and scene history window
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=2
)

# Evaluate Scene 3 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_3_desc,
    scene_number=3
)

🔁 Agent will consider the last 2 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1, 2]
Scene 3 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The comedic tone of the new scene is consistent with the previous scenes. The humor is light-hearted and revolves around the eccentricities of the characters and their interactions.

Comedic Improvement Recommendations:
1. Suggestion: When Earl shows off his latest obsession – an antique lock – he attempts to demonstrate its "unbreakable" feature by challenging Jimmy to pick it. After a series of unsuccessful attempts, Earl takes back the lock only to accidentally lock his own fingers together, much to Jimmy's amusement. Earl, however, insists this is a "safety feature". — This suggestion builds on Earl's eccentric character and his obsession with locks. It also introduces a funny visual gag that aligns well with the comedic style of the sitcom. 
2. Suggestion: Earlier in the series, it was establish

### Enviroment Agent


In [33]:
# Re-initialize the environment agent with the language model client, metadata store, and scene history window
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=2
)

# Evaluate Scene 3 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_3_desc,
    scene_number=3
)

🔁 Environment Agent will consider the last 2 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1, 2]
Scene 3 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from the living room back to Jimmy's Shop is logical. It's plausible that the characters could have left the living room and returned to the shop for any number of reasons. 
- Suggested Transition Setup: Not necessary in this case.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As a nod to Earl's lock-related obsession, there can be a comedic moment where we see Earl trying to pick a particularly stubborn lock on an antique trunk that Jimmy's shop is supposed to fix. This could add a humorous and character-relevant background action to the scene.
- [Suggestion 2]: The sound of a bell jingling every time the shop door opens can be incorporated. This could bring a classic, small-business charm to the shop setting and also serve as a comedic c

### Scene Planner Agent

In [34]:
# Generate a plan for Scene 3 by integrating recommendations from all three agents
scene_3_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=3
)

print(scene_3_plan)

Scene Plan:
Character Goals:
- [Goal 1]: Establish a deeper connection between Earl and Jimmy through a shared moment of fascination with Jimmy's hairpin, highlighting Earl's eccentricity and Jimmy's unconventional interests.
- [Goal 2]: Further develop Earl's sociable personality by having him show genuine interest in Jimmy's unusual plant-care habits, promising to contribute his own eccentric solution.

Comedic Goal:
- [Goal]: Introduce a humorous sequence where Earl accidentally locks his fingers together while demonstrating an "unbreakable" lock, and exacerbates his forgetfulness by using an oversized magnifying glass to inspect the lock, unaware that his glasses are in his pocket.

Environment Detail:
- [Detail]: Incorporate a variety of unique, antique locks and keys displayed around Jimmy's shop, each with their own peculiarities, which Earl is frequently seen tinkering with in the background.

Creative Suggestion:
- [Suggestion]: As the scene unfolds, Earl's attempts to pick th

### Generating Scene 3 Script

In [35]:
# Generate the full script for Scene 3 based on the structured scene plan
scene_3_script = generate_scene(
    client=client,
    scene_plan=scene_3_plan,
    scene_number= 3
)

print(scene_3_script)

INT. JIMMY'S LOCKSMITH SHOP - DAY

A variety of antique locks and keys are displayed around the shop. EARL, an eccentric man in his 60s, is examining a peculiar lock with an oversized magnifying glass, his glasses conspicuously poking out from his pocket. JIMMY, a young, unconventional locksmith, is trimming a bizarre plant with a pair of small scissors. 

EARL:
(Looking through the magnifying glass)
This lock is... (squints) ...fascinating! Just like your hairpin, Jimmy.

[Laugh Track]

JIMMY:
(Chuckles)
You're the only one who'd find a hairpin fascinating, Earl.

EARL:
(Still inspecting)
And you're the only one who'd feed a plant cheese puffs. 

[Laugh Track]

Jimmy shrugs and tosses a cheese puff to the plant.

JIMMY:
(Smiling)
It's an acquired taste. 

EARL:
(Excited)
Wait till I tell you about my compost tea. 

[Laugh Track]

Earl, still peering through the magnifying glass, attempts to demonstrate an "unbreakable" lock, but locks his fingers together instead.

EARL:
(Struggling)


### Adding Metadata to Vector Database

In [36]:
# Store the generated Scene 3 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_3_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 3 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. Hi

## Scene 4 Generation

For scenes 4 and beyond, the 3 previous scenes are retreived.

In [37]:
# Extract the description for Scene 4 from the episode outline
scene_4_desc = extract_scene(episode_outline, 4)
print(scene_4_desc)

Scene 4: "Blast from the Past" Jimmy opens the mysterious package to find a lock from his criminal past, causing him to reminisce. The tone is a mix of nostalgia and humor.


### 🎭 Character Agent


In [38]:
# Re-initialize the character agent with the language model client, metadata store, and total number of scenes
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 4 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_4_desc,
    scene_number=4
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1, 2, 3]
Scene 4 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Jimmy's actions in the planned scene align with his established personality, emotional arc, and speaking style. His humor is consistent with previous scenes, as is his use of a hairpin to open the package. The introduction of a lock from his past speaks to his unconventional and eccentric character. His reminiscing could also add depth to his character, fitting in with the growing acceptance of his quirks seen in Scene 3.

Interaction Improvement Recommendations:
1. Jimmy uses a hairpin to unlock the old lock from his past, and in a moment of nostalgia, shares a funny story about his hairpin adventures to Mrs. Jenkins, the nosy neighbor. — (This interaction fits because it continues the running joke about Jimmy's unconventional use of hairpins as seen in Scenes 1, 2, and 3. It also builds on the friendly

### 🎤 Comedic Agent


In [39]:
# Re-initialize the comedy agent with the language model client, metadata store, and scene history window
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 4 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_4_desc,
    scene_number=4
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1, 2, 3]
Scene 4 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The tone of nostalgic humor in the new scene is consistent with the quirky and light-hearted humor of the previous scenes. The humor is derived from Jimmy's unconventional habits and interactions with his pets and friends, which is a recurring theme.

Comedic Improvement Recommendations:
1. [Suggestion] — As Jimmy opens the package, he pulls out the lock and lets out a surprised, "Oh, wow... my old lock pick." His roommate, Ben, deadpans, "You mean the one you used to break into the school gym to steal the mascot costume?" Jimmy grins sheepishly, "No, the one I used to break into the Principal's office to change my grades. The mascot thing was a completely different lock pick." This addition builds on Jimmy's past as a mischievous character and brings in humor through the unexpected escalation of 

### 🌆 Environment Agent


In [40]:
# Re-initialize the environment agent with the language model client, metadata store, and scene history window
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 4 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_4_desc,
    scene_number=4
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1, 2, 3]
Scene 4 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from Jimmy's Locksmith Shop to Jimmy's House is logical and believable. It's reasonable to assume that after a day at work, Jimmy would return home. The audience would expect this kind of transition in a sitcom.
- Suggested Transition Setup: Not necessary in this case.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As Jimmy enters his house, he absentmindedly tries to pick the lock of his own front door before realizing he doesn't need to, giving a nod to his criminal past. 
- [Suggestion 2]: The mysterious package on the kitchen table is wrapped in an extravagantly colorful paper and a large, flamboyant bow, contrasting with the otherwise mundane surroundings of Jimmy's house. It keeps making a weird ticking noise, causing Jimmy to jump

### 🧠 Scene Planner Agent


In [41]:
# Generate a plan for Scene 4 by integrating recommendations from all three agents
scene_4_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=4
)

print(scene_4_plan)

Scene Plan:
Character Goals:
- [Goal 1] Jimmy must continue to showcase his playful side and resilience by using his hairpin to unlock a mysterious package, sparking nostalgia and a humorous interaction with Mrs. Jenkins. This interaction should further develop their friendly rivalry and showcase more of Jimmy's unconventional skills.
- [Goal 2] Jimmy should engage in playful banter with Martin, revealing more about his past and establishing a strong bond between the two, while also reinforcing the comedic theme of Jimmy's reformed criminal past.

Comedic Goal:
- [Goal] Jimmy should reveal his mischievous past in a comedic manner, escalating the humor by sharing unexpected and increasingly outrageous stories of his criminal exploits. The humor should come from Jimmy's casual recollection of these incidents and his roommate's deadpan reactions.

Environment Detail:
- [Detail] The scene should take place in Jimmy's home, which is decorated with subtle reminders of his past life, like a f

### Generating Scene 4 Script

In [42]:
# Generate the full script for Scene 4 based on the structured scene plan
scene_4_script = generate_scene(
    client=client,
    scene_plan=scene_4_plan,
    scene_number= 4
)

print(scene_4_script)

INT. JIMMY'S APARTMENT - DAY

Jimmy is sitting on the couch, a flamboyantly wrapped package on the coffee table. A FRAMED PHOTOGRAPH of him wearing a stolen mascot costume hangs on the wall. He picks up a hairpin from the table.

JIMMY: (examining the package) Hmm. A ticking sound. Just like old times. 

[Laugh Track]

He begins to pick the lock with the hairpin. Mrs. Jenkins, his elderly neighbor, enters the apartment.

MRS. JENKINS: (raising an eyebrow) Still up to your old tricks, I see.

JIMMY: (grinning) It's like riding a bike, Mrs. J.

[Laugh Track]

Jimmy successfully opens the package to reveal an OLD NOTE. He reads it aloud.

JIMMY: (reading) "Remember the time you stole the zoo monkey?"

[Laugh Track]

Mrs. Jenkins looks horrified. Jimmy chuckles.

JIMMY: (nostalgic) Ah, the good old days.

Martin, Jimmy's roommate, walks in.

MARTIN: (deadpan) I'm afraid to ask. The zoo monkey?

JIMMY: (nodding) Yep. Dressed him up in a suit, taught him to pick pockets. Made a killing at th

### 🗂️ Adding Metadata to Vector Database

In [43]:
# Store the generated Scene 4 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_4_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 4 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. Hi

## Scene 5 Generation

In [44]:
# Extract the description for Scene 5 from the episode outline
scene_5_desc = extract_scene(episode_outline, 5)
print(scene_5_desc)

Scene 5: "Lock of Trouble" Jimmy and Earl try to open the lock, turning the situation into a humorous mess.


### 🎭 Character Agent


In [45]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 5 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_5_desc,
    scene_number=5
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [2, 3, 4]
Scene 5 — Consistency: ✅ Consistent

Explanation:
1. Consistency Verdict (Yes/No)
Yes
2. Short Explanation Why (max 5 lines)
Both Jimmy and Earl behave consistently with their established personalities in this planned scene. Jimmy's unconventional and humorous traits are reflected in the scenario of turning lock opening into a humorous mess. Similarly, Earl's eccentricity and humor are also consistent with the situation. Their actions and dialogue are likely to remain consistent with the traits and relationships established in the past scenes.

Interaction Improvement Recommendations:
1. Jimmy attempts to open the lock with a hairpin, causing Earl to chuckle and joke that he's feeding the lock "lock nutrients" like he does with his plant. — This interaction draws on Jimmy's unconventional use of hairpins and his humorous relationship with Earl from Scene 2. It also references Ji

### 🎤 Comedic Agent


In [46]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 5 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_5_desc,
    scene_number=5
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [2, 3, 4]
Scene 5 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The comedic tone of absurdity and slapstick is consistent with the previous scenes. The characters continue to find themselves in ridiculous situations, and the humor is derived from their unconventional responses and physical comedy.

Comedic Improvement Recommendations:
1. [Suggestion] — In a previous scene, Jimmy showed a tendency to overthink and get lost in his thoughts. Use this character trait here. As Jimmy and Earl struggle with the lock, Jimmy starts giving a lengthy, overly-analytical explanation about the mechanics of locks, meanwhile, Earl has been mindlessly jiggling the lock which suddenly opens. Have Earl cut Jimmy's lengthy explanation with a simple, "It's open." — This not only adds humor through character contrast but also reinforces Jimmy's tendency to overcomplicate things.
2.

### 🌆 Environment Agent


In [47]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 5 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_5_desc,
    scene_number=5
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [2, 3, 4]
Scene 5 — Environment Transition: ❌ Inconsistent

Explanation:
Short Explanation: Without knowing the current scene environment, it's impossible to determine if the transition is logical or not. The previous scenes have followed a logical progression, but the current scene is unknown.
- Suggested Transition Setup: Once the current scene environment is known, a transition setup can be suggested. For example, if the next scene is at a restaurant, a setup could be Jimmy mentioning he's hungry or has a date after leaving his apartment.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: If the next scene is in a restaurant, start the scene with a waiter dropping a tray of dishes, followed by a comedic reaction from the main characters. This adds a touch of humor, sets the location, and provides a natural transition.
- [Suggestion 2]: If th

### 🧠 Scene Planner Agent


In [48]:
# Generate a plan for Scene 5 by integrating recommendations from all three agents
scene_5_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=5
)

print(scene_5_plan)

Scene Plan:
Character Goals:
- [Goal 1]: Develop Jimmy's character by showcasing his tendency to overthink simple situations and his unconventional problem-solving methods, such as using a hairpin to pick a lock.
- [Goal 2]: Highlight Earl's eccentricity and comedic ineptitude through physical comedy, such as accidentally locking his fingers together or hilariously misreading simple instructions.

Comedic Goal:
- [Goal]: Incorporate humor through character contrast and running gags, such as Jimmy's overcomplication of situations and Earl's reading struggles.

Environment Detail:
- [Detail]: If the next scene takes place in a home setting, introduce a malfunctioning coffee maker that the characters attempt to fix, providing a comedic moment and setting the stage for the lock-picking scenario.

Creative Suggestion:
- [Suggestion]: As the scene progresses, Jimmy and Earl's attempts to pick the lock could trigger a series of comedic mishaps. The malfunctioning coffee maker could explode, c

### Generating Scene 5 Script

In [49]:
# Generate the full script for Scene 5 based on the structured scene plan
scene_5_script = generate_scene(
    client=client,
    scene_plan=scene_5_plan,
    scene_number= 5
)

print(scene_5_script)

INT. KITCHEN - MORNING

[EARL, a bumbling yet lovable character, is fumbling with a COFFEE MAKER.]

EARL
(reading instructions)
"Insert the... coffee... into the... filer?"

[Laugh Track]

JIMMY, a neurotic character who overthinks every situation, enters the room.

JIMMY
That's "filter", Earl.

EARL
Oh, right. "Filter". 

[Earl pours coffee grounds directly into the water reservoir.]

[Laugh Track]

JIMMY
No, Earl! The filter, not the water tank!

EARL
Oh... Oops!

[Jimmy sighs and tries to open the coffee maker to fix Earl's mistake. The door is stuck.]

JIMMY
It's locked. Earl, do you have a hairpin?

[Laugh Track]

EARL
(looking confused)
A hairpin? For what, a new hairstyle?

JIMMY
(smirking)
No, Earl, to pick the lock.

[Jimmy takes the hairpin and starts to pick the lock.]

[Earl tries to wipe up the spilled coffee grounds but accidentally glues his fingers together with a sticky sponge.]

[Laugh Track]

EARL
(panicking)
Jimmy, I can't separate my fingers!

JIMMY
(still focused 

### 🗂️ Adding Metadata to Vector Database

In [50]:
# Store the generated Scene 5 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_5_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 5 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. Hi

## Scene 6 Generation

In [51]:
# Extract the description for Scene 6 from the episode outline
scene_6_desc = extract_scene(episode_outline, 6)
print(scene_6_desc)

Scene 6: "Sophie's Arrival" Sophie, Jimmy's estranged daughter, unexpectedly shows up at the shop, creating an awkward and emotional moment.


### 🎭 Character Agent


In [52]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 6 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_6_desc,
    scene_number=6
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [3, 4, 5]
Scene 6 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Based on the past scenes, Jimmy's encounter with his estranged daughter Sophie is consistent with his unconventional personality and emotional arc. His ability to find humor in awkward situations may likely play out in this scene. However, Sophie's personality and speaking style are not described in the past scenes, making it hard to anticipate her behavior.

Interaction Improvement Recommendations:
1. Jimmy, taken aback by Sophie's appearance, instinctively tries to diffuse the tension with humor, cracking a joke about how he was just about to use his hairpin to pick a lock. — (This interaction is consistent with Jimmy's humorous personality and his habit of using humor in uncomfortable situations, as shown in Scene 2 when he jokes about his criminal past. It also includes his running behavior of lock p

### 🎤 Comedic Agent


In [53]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 6 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_6_desc,
    scene_number=6
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [3, 4, 5]
Scene 6 — Comedic Tone: ❌ Inconsistent

Comedic Tone Analysis:
3. Short Explanation: The previous scenes have been characterized by physical comedy and humorous banter, with a focus on Jimmy's past mischief and Earl's misreading of instructions. The introduction of an estranged daughter and the potential for emotional humor is a new element that hasn't been previously established.

Comedic Improvement Recommendations:
1. [Suggestion] - As Sophie enters the shop, she accidentally knocks over a display of car parts due to her clumsiness, a trait she's inherited from Jimmy. This initiates a humorous chain reaction, where Earl frantically tries to catch the falling items, but misreads the situation and instead grabs a bucket of oil, spilling it all over the floor. Sophie, Jimmy, and Earl all end up slipping and sliding around the shop in the oil, bringing physical comedy into the sc

### 🌆 Environment Agent


In [54]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 6 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_6_desc,
    scene_number=6
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [3, 4, 5]
Scene 6 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from the kitchen in Jimmy's apartment to the shop is logical. It's plausible that after spending time in his apartment, Jimmy could go back to his locksmith shop for work or any other reason. 
- Suggested Transition Setup: Not necessary.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As Jimmy enters the shop, a jingling bell over the door signals his arrival. The bell is comically oversized for such a small shop, suggesting that Jimmy has a quirky sense of humor despite his serious profession.
- [Suggestion 2]: The smell of metal and oil is strong, a reminder of the shop's primary function as a locksmith business. There's a small, slightly dented, "World's Best Dad" mug on the counter filled with various keys, adding a touch of irony and

### 🧠 Scene Planner Agent


In [55]:
# Generate a plan for Scene 6 by integrating recommendations from all three agents
scene_6_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=6
)

print(scene_6_plan)

Scene Plan:
Character Goals:
- [Goal 1]: Develop Jimmy's emotional depth by having him react to Sophie's peace offering in a vulnerable way, revealing a side of him we haven't seen before.
- [Goal 2]: Introduce Sophie's mischievous side as she attempts to outdo Jimmy's past antics, strengthening their father-daughter bond and adding depth to her character.

Comedic Goal:
- [Goal]: Incorporate physical comedy through a chain reaction initiated by Sophie's clumsiness, followed by Earl's misinterpretation of the situation, leading to a comedic chaos in the shop.

Environment Detail:
- [Detail]: The locksmith shop is filled with an array of eccentric items that reflect Jimmy's humorous and quirky personality, including an absurdly large bell over the door, and a key-filled "World's Best Dad" mug, adding a layer of irony given his estranged relationship with Sophie.

Creative Suggestion:
- [Suggestion]: As Sophie attempts to outdo Jimmy's past antics, she unwittingly triggers an old prank J

### Generating Scene 6 Script

In [56]:
# Generate the full script for Scene 6 based on the structured scene plan
scene_6_script = generate_scene(
    client=client,
    scene_plan=scene_6_plan,
    scene_number= 6
)

print(scene_6_script)

INT. LOCKSMITH SHOP - DAY

[Sophie enters, causing an absurdly large bell above the door to CLANG loudly.]

SOPHIE: 
Dad, I wanted to... 

[She notices the key-filled "World's Best Dad" mug on the counter.]

SOPHIE: 
...make a peace offering. 

[She presents a box of cheese puffs.]

JIMMY:
[Laugh Track]
Sophie, I can't eat those. They're for my plant.

SOPHIE:
[Laughs]
You're not the only one who can play tricks, Dad. 

[She opens the box and cheese puff dust explodes, covering everyone.]

EARL:
[Laugh Track]
Sophie, why did you blow up the cheese puffs?

SOPHIE: 
I didn't know it would explode!

JIMMY:
[Laugh Track]
That was one of my old pranks, Sophie. 

SOPHIE:
[Smiling]
I guess I did learn from the best, didn't I?

[Jimmy smiles and wipes cheese puff dust off Sophie's face.]

JIMMY: 
You sure did, kiddo. 

[They share a laugh and a warm hug. Earl, standing nearby, wipes cheese puff dust off his face and grumbles.]

EARL:
[Laugh Track]
And I thought this was just a locksmith shop..

### 🗂️ Adding Metadata to Vector Database

In [57]:
# Store the generated Scene 6 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_6_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 6 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. Hi

## Scene 7 Generation

In [58]:
# Extract the description for Scene 7 from the episode outline
scene_7_desc = extract_scene(episode_outline, 7)
print(scene_7_desc)

Scene 7: "Father-Daughter Tensions" Sophie and Jimmy engage in a tense conversation about their past and why she has come. The scene is emotional yet comical due to their differences.


### 🎭 Character Agent


In [59]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 7 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_7_desc,
    scene_number=7
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [4, 5, 6]
Scene 7 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Sophie, as previously established, is good-humored and peace-loving, which is consistent with her engaging in a conversation about their past. Jimmy's prankster side may bring comic relief to the tense conversation, in line with his character. However, his warm and affectionate side towards Sophie, established in Scene 3, will likely shine through during the discussion. There's no clear contradiction based on the previous scenes.

Interaction Improvement Recommendations:
1. Sophie brings up one of Jimmy's past pranks, using it as a metaphor to discuss their strained relationship. Despite the serious topic, they both end up laughing about the absurdity of the prank, reflecting their shared sense of humor. — (This interaction builds on Scene 3, where Sophie's reaction to Jimmy's prank hints at a shared sen

### 🎤 Comedic Agent


In [60]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 7 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_7_desc,
    scene_number=7
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [4, 5, 6]
Scene 7 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The new scene maintains the comedic tone of the previous scenes, using humor to lighten serious or emotional moments. The characters' differences and past experiences are used as a source of comedy, consistent with the humor in previous scenes.

Comedic Improvement Recommendations:
1. [Suggestion] — During the tense conversation, Sophie can try to explain her reason for coming back using a complex metaphor involving her love for cooking. Jimmy, not being a foodie, completely misunderstands her metaphor, thinking she's actually talking about a recipe. The exchange could go something like this:
2. [Suggestion] — As Sophie is trying to express her feelings, Jimmy, being a man of few words, could attempt to respond with a thumbs up or a simple "gotcha." Sophie, frustrated, could snap back:

Comedic Ag

### 🌆 Environment Agent


In [61]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 7 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_7_desc,
    scene_number=7
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [4, 5, 6]
Scene 7 — Environment Transition: ❌ Inconsistent

Explanation:
Short Explanation: The current scene environment is not specified, which makes it impossible to determine if the transition is logical or not. The audience needs to know where the characters are to understand the context of the scene.
- Suggested Transition Setup: Since the last scene was in a locksmith shop, a logical transition could be to a hardware store or back to Jimmy's apartment. If the scene is meant to be in a completely different location, a short setup could involve Jimmy receiving a phone call or text message, prompting him to go to the new location.

Environment Detail Suggestions:
Environment Details Suggestions:
- Suggestion 1: As Jimmy enters the hardware store, he's immediately hit by the strong scent of freshly cut wood and paint fumes. He sneezes comically, drawing the attention of a f

### 🧠 Scene Planner Agent


In [62]:
# Generate a plan for Scene 7 by integrating recommendations from all three agents
scene_7_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=7
)

print(scene_7_plan)

Scene Plan:
Character Goals:
- Sophie utilizes a shared past experience, specifically one of Jimmy's pranks, to humorously address their strained relationship, highlighting their shared sense of humor despite their differences.
- Jimmy uses his knack for lock-picking as a metaphor to explain his efforts in mending their relationship, showing his humor and resourcefulness amidst a serious situation.

Comedic Goal:
- During their heartfelt conversation, Sophie uses a cooking metaphor to explain her decision to return, leading to a humorous misunderstanding by Jimmy, who thinks she's literally talking about cooking stew.

Environment Detail:
- As the scene unfolds in Jimmy's apartment, a recurring sound of a neighbor's pet dog howling along to the neighbor's tuba practice can be heard, adding to the overall comedic undertone of the scene.

Creative Suggestion:
- As the conversation heats up, Sophie accidentally triggers one of Jimmy's leftover pranks from the past, causing a mini-chaos in

### Generating Scene 7 Script

In [63]:
# Generate the full script for Scene 7 based on the structured scene plan
scene_7_script = generate_scene(
    client=client,
    scene_plan=scene_7_plan,
    scene_number= 7
)

print(scene_7_script)

INT. JIMMY'S APARTMENT - DAY

[The sound of a tuba followed by a howling dog can be heard from the neighbor's apartment.]

Sophie and Jimmy sit on the couch, surrounded by a mess of old photos and memorabilia.

SOPHIE:
Remember when you rigged the front door to drench whoever opened it with water? 

JIMMY:
(laughs)
Yeah, I didn't think you'd be the first one home that day.

[Laugh Track]

Sophie picks up an old lock and hands it to Jimmy.

SOPHIE:
And when you picked the lock to the principal's office?

JIMMY:
(smiling)
Oh, that was a classic.

SOPHIE:
You were always good at getting through locks, Jimmy. Even the ones you weren't supposed to.

JIMMY:
(laughs)
Well, some locks are meant to be picked, Sophie. Like the one between us.

[Laugh Track]

SOPHIE:
I guess that's why I'm back, Jimmy. I want to try and pick that lock again. Like cooking a stew, you know? It takes time, and patience, and the right ingredients.

JIMMY:
(confused)
Wait, are we still talking about us or do you want 

### 🗂️ Adding Metadata to Vector Database

In [64]:
# Store the generated Scene 7 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_7_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 7 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. Hi

## Scene 8 Generation

In [65]:
# Extract the description for Scene 8 from the episode outline
scene_8_desc = extract_scene(episode_outline, 8)
print(scene_8_desc)

Scene 8: "Earl's Intervention" Earl tries to mediate the situation between Sophie and Jimmy, leading to a funny and awkward moment.


### 🎭 Character Agent


In [66]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 8 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_8_desc,
    scene_number=8
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [5, 6, 7]
Scene 8 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Earl's intervention aligns with his endearing clumsiness and eccentricity, as well as his friendly relationship with Jimmy. His attempt to mediate could result in humorous mishaps, which is consistent with his character. Sophie's willingness to engage in the intervention is consistent with her forgiving, reconciliatory nature and her desire to improve her relationship with Jimmy. Jimmy's participation aligns with his evolving emotional arc, which shows him valuing relationships more.

Interaction Improvement Recommendations:
1. Earl, in his clumsy yet endearing way, tries to explain to Sophie how much Jimmy's pranks mean to him, accidentally setting off another of Jimmy's old pranks in the process. This could lead to a funny moment of chaos that lightens the tension between Sophie and Jimmy — (This inter

### 🎤 Comedic Agent


In [67]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 8 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_8_desc,
    scene_number=8
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [5, 6, 7]
Scene 8 — Comedic Tone: ❌ Inconsistent

Comedic Tone Analysis:
3. Short Explanation: The awkward humor in this scene is consistent with the comedic tone of the previous scenes, which also revolve around humorous misunderstandings and unexpected chaos.

Comedic Improvement Recommendations:
1. [Suggestion] — Earl, in an attempt to demonstrate neutrality, tries to sit exactly in the middle of Sophie and Jimmy but ends up sitting on a small, shaky stool that collapses under his weight. This leads to a humorous moment where Earl lands on a whoopee cushion that Jimmy had sneakily placed on the floor earlier in the episode. — (This suggestion builds on the physical comedy and slapstick humor that has been present in prior scenes, while also incorporating a running joke of Jimmy's love for practical jokes.)
2. [Suggestion] — As Earl tries to mediate, he unknowingly uses a string of idio

### 🌆 Environment Agent


In [68]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 8 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_8_desc,
    scene_number=8
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [5, 6, 7]
Scene 8 — Environment Transition: ❌ Inconsistent

Explanation:
Short Explanation: There is no clear connection between Jimmy's apartment and Earl's intervention. The audience might be confused about how and why the characters ended up at an intervention from an apartment without any prior setup or context.
- Suggested Transition Setup: A scene could be added where Jimmy finds a letter in his apartment, which is an invitation to Earl's intervention. This would provide a logical transition and also add a layer of surprise and intrigue.

Environment Detail Suggestions:
Environment Details Suggestions:
- Suggestion 1: As they enter Earl's intervention, the characters are greeted by a peculiar sight: an 'Intervention' banner crudely made from duct tape and printer paper, hanging lopsidedly across the room. This silly detail can provide a light, comedic moment to ease the 

### 🧠 Scene Planner Agent


In [69]:
# Generate a plan for Scene 8 by integrating recommendations from all three agents
scene_8_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=8
)

print(scene_8_plan)

Scene Plan:
Character Goals:
- Earl must inadvertently trigger one of Jimmy's old pranks while trying to explain the importance of Jimmy's prankster nature to Sophie, thereby showcasing his clumsy yet lovable persona.
- In an effort to mediate the situation between Sophie and Jimmy, Earl must use incorrect pronunciations or idioms, leading Sophie to gently correct him, further showcasing her forgiving nature and advancing their father-daughter bond.

Comedic Goal:
- Incorporate physical comedy by having Earl attempt to sit equidistant between Sophie and Jimmy on a precarious stool, leading to a comical fall on a strategically placed whoopee cushion by Jimmy.

Environment Detail:
- Earl's intervention room should have a noticeable, whimsical element such as an intervention banner made from mismatched materials (e.g., old Christmas lights and a bed sheet). This detail should underscore the awkward but humorous tone of the scene.

Creative Suggestion:
- The scene could progress with Earl 

### Generating Scene 8 Script

In [70]:
# Generate the full script for Scene 8 based on the structured scene plan
scene_8_script = generate_scene(
    client=client,
    scene_plan=scene_8_plan,
    scene_number= 8
)

print(scene_8_script)

INT. EARL'S INTERVENTION ROOM - DAY

The room is haphazardly adorned with an intervention banner made of old Christmas lights and a bed sheet. EARL, SOPHIE, and JIMMY are present. Earl is trying to sit equidistant between Sophie and Jimmy on a precarious stool.

EARL
(awkwardly)
You see, Sophie, your uncle Jimmy's prankster nature is a key part of our family's... er... charisma.

[Laugh Track]

Earl adjusts on the stool, but it wobbles, causing him to sit on a strategically placed whoopee cushion.

[Laugh Track]

SOPHIE
(smiling)
Dad, it's "character," not "charisma."

EARL
(embarrassed)
Right, right, character.

[Laugh Track]

Earl picks up a book from a nearby table.

EARL
(cont'd)
Now, Jimmy, remember this old prank book?

JIMMY
(laughs)
You mean the one with the magic tricks?

EARL
Exactly!

Earl tries to perform a trick, but clumsily opens a secret compartment in an old chest instead.

[Laugh Track]

Sophie and Jimmy both gasp in surprise. Earl pulls out an old photo album.

SOPHI

### 🗂️ Adding Metadata to Vector Database

In [71]:
# Store the generated Scene 8 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_8_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 8 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. Hi

## Scene 9 Generation

In [72]:
# Extract the description for Scene 9 from the episode outline
scene_9_desc = extract_scene(episode_outline, 9)
print(scene_9_desc)

Scene 9: "A Locksmith's Lesson" Jimmy reluctantly agrees to teach Sophie some locksmithing skills, leading to a series of funny and touching moments.


### 🎭 Character Agent


In [73]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 9 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_9_desc,
    scene_number=9
)


🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [6, 7, 8]
Scene 9 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Both Jimmy and Sophie's behaviors align with their established character profiles. Jimmy's willingness to teach Sophie locksmithing skills shows his warm-hearted nature, while the humor that ensues from the situation aligns with his prankster personality. Sophie's desire to learn from Jimmy aligns with her peacemaker and open-minded traits. Her ability to partake in the humor aligns with her good-humored personality. Their actions and dialogue are logical based on past scenes, and there are no contradictions or missing motivations.

Interaction Improvement Recommendations:
1. Jimmy pulls a prank by rigging a lock to squirt ink when Sophie attempts to pick it. Sophie, foreseeing this, switches the locks before Jimmy notices, causing him to be the recipient of his own prank. — (This interaction fits as it 

### 🎤 Comedic Agent


In [74]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 9 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_9_desc,
    scene_number=9
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [6, 7, 8]
Scene 9 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The comedic tone in the new scene is consistent with the previous scenes. It continues to blend humor with touching moments, maintaining the balance between comedy and sentimentality. The pranks, Jimmy's locksmithing skills, and Earl's mispronunciations are all present, keeping the humor consistent.

Comedic Improvement Recommendations:
1. [Suggestion] — As Jimmy is showing Sophie the basics of locksmithing, she could accidentally lock them both inside the shop due to misunderstanding his instructions. Jimmy, trying to keep his cool, guides her through unlocking the door from the inside. This could lead to a funny dialogue exchange where Jimmy exclaims, "You're the first person I've ever met who needs a locksmith to get out of a locksmith's shop!" — (This situation builds on the humor of Sophie's 

### 🌆 Environment Agent


In [75]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 9 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_9_desc,
    scene_number=9
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [6, 7, 8]
Scene 9 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition back to the Locksmith Shop is logical as it was one of the previous locations. The characters could have had a reason to return there, such as needing to pick up a key or discuss something with the locksmith.
- Suggested Transition Setup: Not necessary, but to make it more seamless, a character could mention needing to return to the Locksmith Shop for a specific reason while they are in Earl's Intervention Room.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As a comedic touch, the locksmith shop could have a "Wall of Shame" that displays the most bizarre items people have accidentally locked themselves out of - a birdcage, a mini fridge, a diary, etc.
- [Suggestion 2]: The constant, almost rhythmic sound of metal on metal as the locksmi

### 🧠 Scene Planner Agent


In [76]:
# Generate a plan for Scene 9 by integrating recommendations from all three agents
scene_9_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=9
)

print(scene_9_plan)

Scene Plan:
Character Goals:
- [Goal 1] Sophie needs to demonstrate growth by outsmarting Jimmy's prank, showing her adaptability and learning from past experiences.
- [Goal 2] Jimmy should convey a valuable lesson about patience and precision to Sophie, using a metaphor related to their shared history, in order to bond with her and showcase his warm-hearted nature.

Comedic Goal:
- [Goal] Incorporate a humorous misunderstanding between Jimmy and Sophie, where Sophie locks them inside the locksmith shop and Jimmy tries to guide her to unlock it, leading to a funny dialogue exchange.

Environment Detail:
- [Detail] Introduce a "Customer Lock Fails" display in the shop, featuring the most peculiar items customers have locked themselves out of, with Jimmy adding to the collection throughout the series.

Creative Suggestion:
- [Suggestion] After Sophie outsmarts Jimmy's prank, they get locked inside the shop. As they work to escape, Sophie begins comparing lock picks to shoe styles, leadin

### Generating Scene 9 Script

In [77]:
# Generate the full script for Scene 9 based on the structured scene plan
scene_9_script = generate_scene(
    client=client,
    scene_plan=scene_9_plan,
    scene_number= 9
)

print(scene_9_script)

INT. LOCKSMITH SHOP - DAY

Sophie is carefully examining a lock, eyebrows furrowed. Jimmy is watching her with an amused smirk.

SOPHIE: 
So, this is your grand prank, Jimmy? Switching the locks?

JIMMY: 
(Chuckles)
Well, aren't you a quick one, Soph?

[Laugh Track]

Sophie successfully unlocks the new lock, grinning triumphantly at Jimmy.

SOPHIE: 
Who says you can't teach old dogs new tricks? 

Jimmy looks surprised but then gives a warm laugh.

JIMMY: 
(Laughs)
You got me there, Soph. 

[Laugh Track]

Jimmy then turns serious and points at the lock.

JIMMY:
But remember, Soph. Picking a lock is like fishing. You've got to be patient, precise, and know when to reel in.

Sophie nods, understanding the metaphor. Suddenly, she hears a click behind her. They're locked in.

SOPHIE: 
Oh... seems like I've locked us in.

[Laugh Track]

Jimmy sighs but then starts laughing. Sophie joins in, both of them in a fit of giggles.

JIMMY: 
(Laughs)
Alright, Soph. Try to unlock us now.

Sophie appro

### 🗂️ Adding Metadata to Vector Database

In [78]:
# Store the generated Scene 9 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_9_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 9 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. Hi

## Scene 10 Generation

In [79]:
# Extract the description for Scene 10 from the episode outline
scene_10_desc = extract_scene(episode_outline, 10)
print(scene_10_desc)

Scene 10: "Earl's Dilemma" Earl receives a call about a personal issue, adding another layer of chaos to the day. The tone is comical and frantic.


### 🎭 Character Agent


In [80]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 10 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_10_desc,
    scene_number=10
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [7, 8, 9]
Scene 10 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: The proposed scene is consistent with Earl's character as established in the previous scenes. His personality traits of being clumsy and humorous are maintained, with the suggestion of a personal issue adding an element of chaos which is characteristic of Earl's life. The tone of the scene being comical and frantic is also in line with Earl's speaking style and quirks, such as his humorous commentary. No contradictions are identified based on past scenes.

Interaction Improvement Recommendations:
1. When Earl receives the call about his personal issue, have Jimmy be the one who answers the phone and hands it to Earl. As he does so, he accidentally spills coffee on Earl, referencing their shared clumsiness from Scene 1. This interaction builds on their established dynamic and Earl's clumsiness, while als

### 🎤 Comedic Agent


In [81]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 10 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_10_desc,
    scene_number=10
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [7, 8, 9]
Scene 10 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The frantic comedy tone in the new scene is consistent with the previous scenes, which also have a comedic tone. The humor is derived from the characters' interactions and their reactions to unexpected situations, which is a common thread in all the scenes.

Comedic Improvement Recommendations:
1. [Suggestion] — As Earl takes the call, have him accidentally mix up his conversations - responding to personal issues with office lingo and vice versa. Earl could say something like, "Yes, I understand your concern about the plumbing issue at home, but have you considered synergizing our resources and thinking outside the box?" This not only highlights the chaos of the situation but also reinforces Earl's character as a workaholic who struggles to separate his work and personal life. — (This leverages a

### 🌆 Environment Agent


In [82]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 10 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_10_desc,
    scene_number=10
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [7, 8, 9]
Scene 10 — Environment Transition: ❌ Inconsistent

Explanation:
Short Explanation: The current scene environment is not specified, which makes it impossible to determine if the transition is logical or not. The audience needs to know where the characters are to understand the context of the scene.
- Suggested Transition Setup: After leaving the locksmith shop, Jimmy could receive a phone call from another character, inviting him to a new location. This would provide a logical transition to the next scene, whether it's at a restaurant, another character's home, or a public place like a park.

Environment Detail Suggestions:
Environment Details Suggestions:
- Suggestion 1: As Jimmy steps into the new location, a bustling Italian restaurant, he gets hit with the strong aroma of garlic and tomato sauce. In the background, a waiter can be seen struggling to balance a towe

### 🧠 Scene Planner Agent



In [83]:
# Generate a plan for Scene 10 by integrating recommendations from all three agents
scene_10_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=10
)

print(scene_10_plan)

Scene Plan:
Character Goals:
- [Goal 1] Incorporate a situation where Jimmy answers Earl's personal call, accidentally creating a humorous and chaotic scenario by spilling coffee on Earl, further emphasizing their shared clumsiness trait.
- [Goal 2] Introduce Sophie into the scene at an unexpected moment, leading to Earl's comedic mispronunciation, and reinforcing their supportive friendship as they navigate through the ensuing chaos.

Comedic Goal:
- [Goal] Introduce a comedic mix-up where Earl, while trying to handle his personal call and a work issue simultaneously, ends up confusing both contexts, leading to a comedic interplay of office jargon in a personal situation, and further cementing his inability to compartmentalize.

Environment Detail:
- [Detail] Set the scene in an Italian restaurant filled with lively chatter, clinking cutlery, and a visibly flustered waiter trying to navigate through the crowded space with a stack of pizza boxes, adding to the overall chaos.

Creative 

### Generating Scene 10 Script

In [84]:
# Generate the full script for Scene 10 based on the structured scene plan
scene_10_script = generate_scene(
    client=client,
    scene_plan=scene_10_plan,
    scene_number=10
)

print(scene_10_script)

INT. ITALIAN RESTAURANT - DAY

Earl, a middle-aged man in a business suit, is seated at a table, nervously glancing at his ringing phone. Jimmy, Earl's clumsy but well-meaning coworker, is sitting across from him.

EARL:
(looking at the caller ID)
It's a personal call, but I've got this copier issue at work I need to handle. Jimmy, can you--

Before Earl can finish his sentence, Jimmy reaches for the phone but accidentally knocks over Earl's coffee.

[Laugh Track]

JIMMY:
Oops! Sorry, Earl. 

Jimmy hands the phone to Earl while trying to mop up the spilled coffee with napkins.

EARL:
(into the phone, flustered)
Hello, darling... No, no, I'm not in the office. I'm in an Italian restaurant... No, not a meeting. Just a minor coffee spill.

Earl tries to give Jimmy a stern look but ends up with coffee on his shirt.

[Laugh Track]

Suddenly, Sophie, their level-headed coworker, appears at the table with a stack of pizza boxes.

SOPHIE:
(surprised)
Earl, why are you covered in coffee? 

EARL

### 🗂️ Adding Metadata to Vector Database


In [85]:
# Store the generated Scene 10 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_10_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 10 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. H

## 🎬 Scene 11 Generation


In [86]:
# Extract the description for Scene 11 from the episode outline
scene_11_desc = extract_scene(episode_outline, 11)
print(scene_11_desc)

Scene 11: "Locksmithing Mishaps" Sophie struggles with the locksmithing tasks, leading to hilarious blunders and bonding moments with Jimmy.


### 🎭 Character Agent


In [87]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 11 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_11_desc,
    scene_number=11
)


🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [8, 9, 10]
Scene 11 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Sophie's struggle with locksmithing tasks aligns with her adaptability trait and the humorous approach she takes to challenges, as seen in her previous scenes. Her bonding with Jimmy is consistent with their established relationship, with Jimmy acting as a mentor and appreciating humor. Sophie's blunders also align with the sitcom's comedic tone.

Interaction Improvement Recommendations:
1. Sophie, struggling with a particularly tricky lock, wittily comments, "It's like trying to find a matching pair of socks in the dark!" This brings laughter from Jimmy, who then steps in to guide her through the process. — This interaction fits because it showcases Sophie's humor (as seen in Scene 2 where she compares lock picks to shoe styles) and Jimmy's patient and precise nature (also seen in Scene 2 when he advi

### 🎤 Comedic Agent


In [88]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 11 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_11_desc,
    scene_number=11
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [8, 9, 10]
Scene 11 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The comedic tone in the new scene is consistent with the previous scenes. It maintains the light-hearted and playful humor, focusing on character interactions and humorous mishaps.

Comedic Improvement Recommendations:
1. [Suggestion] — As Sophie struggles with the locksmithing tasks, she accidentally locks herself inside the room. In a panic, she calls out to Jimmy for help but when he arrives, he realizes he left his glasses in the other room and can't see the lock clearly. They then attempt a comedic back-and-forth where Sophie tries to guide Jimmy's blurry vision to unlock the door. — (This builds on a prior scene where Jimmy is revealed to be nearsighted and often forgets his glasses. It also provides a humorous situation that naturally arises from Sophie's locksmithing mishaps and Jimmy's 

### 🌆 Environment Agent


In [89]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 11 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_11_desc,
    scene_number=11
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [8, 9, 10]
Scene 11 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from the Italian Restaurant back to the Locksmith Shop is logical. It's possible that after the intervention and a meal at the restaurant, the characters needed to return to the locksmith shop for a reason related to the plot, such as picking up a key or dealing with a lock-related issue.
- Suggested Transition Setup: Not necessary in this case.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As one of the characters enters the shop, they accidentally knock over a display of ornate, antique locks, causing a loud clatter. This can serve as a natural and funny way to introduce the setting and also potentially lead into a comedic situation where the character has to clumsily pick them up.
- [Suggestion 2]: The distinct metallic scent of th

### 🧠 Scene Planner Agent


In [90]:
# Generate a plan for Scene 11 by integrating recommendations from all three agents
scene_11_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=11
)

print(scene_11_plan)

Scene Plan:
Character Goals:
- [Goal 1]: Intensify Sophie's humorous approach to locksmithing, emphasizing her witty nature, while showcasing Jimmy's patient and meticulous personality as he assists her.
- [Goal 2]: Highlight Sophie's ability to adapt and solve problems, even in chaotic situations, and reinforce Jimmy's clumsy but endearing character through a shared cleaning task.

Comedic Goal:
- [Goal]: Incorporate humor through an amusing locksmithing mishap where Sophie locks herself in a room and Jimmy, forgetting his glasses, struggles to unlock the door, leading to a hilarious guided unlocking attempt.

Environment Detail:
- [Detail]: The shop is cluttered with a variety of locksmithing tools and unique keys, creating an atmosphere of organized chaos that mirrors the characters' personalities and their humorous misadventures.

Creative Suggestion:
- [Suggestion]: The scene could progress by having Sophie and Jimmy discover an old, intricate lock that they decide to tackle toget

### ✍️ Generating Scene 11 Script

In [91]:
# Generate the full script for Scene 11 based on the structured scene plan
scene_11_script = generate_scene(
    client=client,
    scene_plan=scene_11_plan,
    scene_number=11
)

print(scene_11_script)

INT. LOCKSMITH SHOP - DAY

Sophie, a quirky and sharp-witted locksmith, is rummaging through a pile of tools. Jimmy, a patient and meticulous assistant, is tidying up a workstation. 

SOPHIE:
(brandishing a tool)
Ah, the perfect tool for the job!

JIMMY:
(grinning)
Sophie, that's a pizza cutter.

[Laugh Track]

Sophie looks at the tool, laughs, and tosses it aside. She pulls out an old, intricate lock.

SOPHIE:
Wow, look at this treasure chest lock! Let's tackle this one together, Jimmy.

They both sit down at the workstation. Sophie starts poking at the lock but accidentally triggers it, locking herself inside a makeshift cage.

SOPHIE:
(blushing)
Well, this is a fine mess I've got us into.

[Laugh Track]

Jimmy chuckles and starts looking for his glasses. He pats down his pockets and then his head, realizing he left them at home.

JIMMY:
(rubbing his eyes)
Sophie, I forgot my glasses. Can you guide me through unlocking this?

SOPHIE:
(smirking)
You're lucky I'm a problem solver, Jimm

### 🗂️ Adding Metadata to Vector Database


In [92]:
# Store the generated Scene 11 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_11_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 11 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. H

## 🎬 Scene 12 Generation


In [93]:
# Extract the description for Scene 12 from the episode outline
scene_12_desc = extract_scene(episode_outline, 12)
print(scene_12_desc)

Scene 12: "Earl's Solution" Earl comes up with a humorous and unconventional solution to his problem. The scene is light-hearted and silly.


### 🎭 Character Agent


In [94]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 12 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_12_desc,
    scene_number=12
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [9, 10, 11]
Scene 12 — Consistency: ✅ Consistent

Explanation:
1. Consistency Verdict: Yes
2. Earl's behavior in Scene 12, where he comes up with a humorous and unconventional solution to his problem, is consistent with his established personality traits, emotional arc, and speaking style. His humor is reflected in the solution he proposes, his explanatory speaking style is likely used in presenting his solution, and his appreciative trait could be seen in his reaction to his coworkers' responses. No contradictions are identified based on the last three scenes.

Interaction Improvement Recommendations:
1. Earl mispronounces a common idiom while explaining his unconventional solution, leading to a humorous misunderstanding among the team. — (This interaction fits because, as established in Scene 2, Earl often finds himself in the role of an explainer and mispronouncing idioms is one of his

### 🎤 Comedic Agent


In [95]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 12 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_12_desc,
    scene_number=12
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [9, 10, 11]
Scene 12 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The tone of light-hearted silliness is consistent with the previous scenes, which also feature humor derived from character quirks, mishaps, and playful banter. The humor in this scene is in line with the established comedic style of the sitcom.

Comedic Improvement Recommendations:
1. [Suggestion] — As Earl is thinking up his unconventional solution, he could be seen trying out a series of ridiculous hats (a chef's hat, a detective cap, a magician's hat) as he believes different hats help him think differently. Each time he changes his hat, he comes up with a more absurd idea. This would add a visual humor to the scene and also play on Earl's quirky belief system that has been established in previous episodes. — (In Episode 4, Earl mentioned that he always wears his 'thinking cap' when he need

### 🌆 Environment Agent


In [96]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 12 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_12_desc,
    scene_number=12
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [9, 10, 11]
Scene 12 — Environment Transition: ❌ Inconsistent

Explanation:
Short Explanation: There is no clear connection between a locksmith shop and a public library. The transition from the locksmith shop to the Italian restaurant and back makes sense if the characters were getting lunch, but the sudden shift to a public library is jarring without context.
- Suggested Transition Setup: One of the characters could mention they need to research a rare lock model or find a book on locksmithing techniques. This would provide a logical reason for the characters to visit the library next.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As the characters enter the library, they are shushed by an eccentric librarian who is overly enthusiastic about quietness and seems to have an uncanny knack for appearing whenever a character speaks too loudly

### 🧠 Scene Planner Agent


In [97]:
# Generate a plan for Scene 12 by integrating recommendations from all three agents
scene_12_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=12
)

print(scene_12_plan)

Scene Plan:
Character Goals:
- [Goal 1]: Maintain Earl's idiosyncratic charm by having him misinterpret a common saying, leading to a funny misunderstanding. This adds depth to Earl's character and provides comic relief.
- [Goal 2]: Develop Earl's character further by showcasing his appreciation for Jimmy's chaotic contributions to their unconventional solution, reinforcing Earl's growth in understanding and tolerance.

Comedic Goal:
- [Goal]: Incorporate visual humor by having Earl try on a variety of hats, each sillier than the last, as part of his "thinking process" for the unconventional solution. Additionally, make Earl's final solution hilariously absurd by involving his pet hamster, Mr. Nibbles, playing on the established running gag of Earl's over-reliance on his hamster.

Environment Detail:
- [Detail]: The library setting should be enhanced with the presence of a peculiarly quiet-seeking librarian who uncannily appears whenever the noise level rises, adding to the comedic ten

### ✍️ Generating Scene 12 Script

In [98]:
# Generate the full script for Scene 12 based on the structured scene plan
scene_12_script = generate_scene(
    client=client,
    scene_plan=scene_12_plan,
    scene_number=12
)

print(scene_12_script)

INT. LIBRARY - DAY

Earl, a lanky, eccentric character, is hunched over a table with Jimmy, a lovable chaos-magnet. They're engrossed in examining a RARE LOCK MODEL. A variety of SILLY HATS are strewn across the table.

EARL
Well, Jimmy, if we're going to crack this, I need my thinking hats.

Earl puts on a FEZ. Jimmy raises an eyebrow but says nothing.

[Laugh Track]

Earl
The way I see it, we need an unconventional solution. Something... out of the box.

Jimmy, not missing a beat, picks up a FRUIT HAT and hands it to Earl.

JIMMY
How about something out of the fruit bowl instead?

[Laugh Track]

Earl chuckles and swaps the fez for the fruit hat. He looks thoughtful.

EARL
Actually, I've got it! We'll use Mr. Nibbles!

Jimmy looks at Earl, disbelief painted on his face.

JIMMY
Your hamster? Earl, that's... that's not just out of the box. That's out of the pet store!

[Laugh Track]

Earl grins, about to explain, when a stern LIBRARIAN appears, shushing them.

LIBRARIAN
This is a librar

### 🗂️ Adding Metadata to Vector Database


In [99]:
# Store the generated Scene 12 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_12_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 12 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. H

## 🎬 Scene 13 Generation


In [100]:
# Extract the description for Scene 13 from the episode outline
scene_13_desc = extract_scene(episode_outline, 13)
print(scene_13_desc)

Scene 13: "Heart-to-Heart" Sophie and Jimmy have a heartfelt conversation about their relationship while working on locks. The tone is emotional and sincere.


### 🎭 Character Agent


In [101]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 13 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_13_desc,
    scene_number=13
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [10, 11, 12]
Scene 13 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Sophie's emotional arc, from prank victim to rescuer, shows her increasing closeness with Jimmy. A heart-to-heart conversation is a logical next step in their relationship. It also aligns with Jimmy's arc, as he's shown to be adaptable and good-natured. Sophie's playful speaking style could add a light-hearted touch to the emotional conversation, while their shared locksmith work provides a consistent setting.

Interaction Improvement Recommendations:
1. Sophie, while working on a lock, jokingly asks Jimmy if he brought his glasses this time, referring to the time he forgot them while trying to unlock the cage she was trapped in (Scene 2). This interaction not only provides a light-hearted moment in their serious conversation but also reminds them of a shared past experience that brought them closer 

### 🎤 Comedic Agent


In [102]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 13 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_13_desc,
    scene_number=13
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [10, 11, 12]
Scene 13 — Comedic Tone: ❌ Inconsistent

Comedic Tone Analysis:
3. Short Explanation: The dominant tone in the new scene is emotional and sincere, which is a departure from the comedic tone of the previous scenes. The previous scenes were characterized by humor derived from physical comedy, quirky character traits, and situational comedy.

Comedic Improvement Recommendations:
1. [Suggestion] — As Sophie and Jimmy work on the locks, they both struggle with their respective locks, resulting in a light-hearted competition. Sophie boasts, "You know, I'm a natural at this. I'd probably be a master locksmith by now if I wasn't so into marketing." Jimmy retorts, "Oh yeah? Well, I'd probably be a...lock...smith...too...if I wasn't so into...uh..." Sophie interrupts, "Into forgetting your punchlines, Jimmy?" — (This suggestion builds on the running joke of Jimmy's tendency to forget h

### 🌆 Environment Agent


In [103]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 13 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_13_desc,
    scene_number=13
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [10, 11, 12]
Scene 13 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The characters were previously at a locksmith shop, then they went to a library, and now they are back at the locksmith shop. This is a logical transition as they could have needed to return to the locksmith shop for a variety of reasons, such as picking up a key they had copied or fixing a lock issue they discovered at the library.
- Suggested Transition Setup: Not necessary in this case.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As the characters enter the locksmith shop, they're greeted by the distinct metallic aroma of freshly cut keys. One character could humorously comment, "Ah, the sweet smell of security."
- [Suggestion 2]: The locksmith could be seen in the background, struggling with an unusually large and ornate lock, giving up in f

### 🧠 Scene Planner Agent


In [104]:
# Generate a plan for Scene 13 by integrating recommendations from all three agents
scene_13_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=13
)

print(scene_13_plan)

Scene Plan:
Character Goals:
- [Goal 1] Develop a dialogue where Sophie playfully teases Jimmy about his past mishaps, including forgetting his glasses and spilling coffee, reinforcing their shared history and highlighting Sophie's humorous nature.
- [Goal 2] Enable Jimmy to express sincere gratitude towards Sophie for her patience, showcasing his growth and adaptability throughout their shared experiences. Sophie should reciprocate with words of appreciation for Jimmy's meticulous nature.

Comedic Goal:
- [Goal] Incorporate a comedic competition between Sophie and Jimmy as they struggle with their locks. Use Jimmy's tendency to forget punchlines as a running gag, and introduce a surprise element with Sophie pulling out a rubber chicken instead of a tissue, referring back to their shared history.

Environment Detail:
- [Detail] The locksmith shop is cluttered with keys and locks of all shapes and sizes, hanging from the ceiling and walls, creating a visually interesting and chaotic bac

### ✍️ Generating Scene 13 Script

In [105]:
# Generate the full script for Scene 13 based on the structured scene plan
scene_13_script = generate_scene(
    client=client,
    scene_plan=scene_13_plan,
    scene_number=13
)

print(scene_13_script)

INT. LOCKSMITH SHOP - DAY

Sophie and Jimmy stand among the cluttered assortment of keys and locks. The shop's parrot squawks, "Locked out again, eh?" Sophie grins at Jimmy.

SOPHIE:
Remember when you locked yourself out of your apartment because you thought your glasses were a lock pick?

Jimmy chuckles nervously, adjusting his glasses.

JIMMY:
Ah, yes. And remember when you thought a cup of coffee could open a jammed door?

[Laugh Track]

Sophie laughs, flipping a lock in her hand.

SOPHIE:
Well, it was worth a shot! You never know until you try, right?

Jimmy nods, holding his own lock.

JIMMY:
True. And thanks for always trying with me, Sophie.

Sophie smiles, patting Jimmy on the back.

SOPHIE:
And thank you for your meticulous nature. Without you, I'd probably be carrying around a bucket of coffee and a rubber chicken for emergencies.

[Laugh Track]

Sophie reaches into her bag to pull out a tissue, but instead pulls out a rubber chicken. Jimmy bursts into laughter.

JIMMY:
Oh, n

### 🗂️ Adding Metadata to Vector Database


In [106]:
# Store the generated Scene 13 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_13_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 13 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. H

## 🎬 Scene 14 Generation


In [107]:
# Extract the description for Scene 14 from the episode outline
scene_14_desc = extract_scene(episode_outline, 14)
print(scene_14_desc)

Scene 14: "The Open Lock" Jimmy and Sophie manage to open the lock from the mysterious package, leading to a surprising revelation. The atmosphere is suspenseful and funny.


### 🎭 Character Agent


In [108]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 14 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_14_desc,
    scene_number=14
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [11, 12, 13]
Scene 14 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Both Jimmy and Sophie are acting in line with their established personality traits and emotional arcs. Jimmy, with his meticulousness, likely played a key role in opening the lock, while Sophie, with her adventurous spirit, would have added an element of humor and surprise to the suspenseful scene. Their actions and dialogue are expected to be consistent with their respective speaking styles and the nature of their relationship, which has been steadily strengthening over the previous scenes.

Interaction Improvement Recommendations:
1. As Jimmy and Sophie work together to open the lock, Jimmy should initially struggle because he can't find his glasses. Sophie, in her usual lighthearted manner, can hand him a pair of sunglasses instead, referencing their past interactions (Scene 1). This interaction w

### 🎤 Comedic Agent


In [109]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 14 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_14_desc,
    scene_number=14
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [11, 12, 13]
Scene 14 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The suspenseful humor in the current scene aligns with the playful banter and unexpected comedic twists found in previous scenes. The characters' quirky personalities and their unconventional problem-solving methods continue to drive the humor.

Comedic Improvement Recommendations:
1. [Suggestion] — As Jimmy and Sophie struggle to open the lock, Jimmy can say, "This lock is tougher than my mom's meatloaf!" to which Sophie responds, "That's because your mom uses the meatloaf recipe from the cookbook we found in that other mysterious package!" This joke not only builds on the suspense of the scene, but it also subtly references a previous episode where they found a cookbook in a mysterious package, hence creating a light running joke. This adds a layer of humor and continuity to the scene.
2. [S

### 🌆 Environment Agent


In [110]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 14 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_14_desc,
    scene_number=14
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [11, 12, 13]
Scene 14 — Environment Transition: ❌ Inconsistent

Explanation:
Short Explanation: Without knowing the current scene environment, it's impossible to determine whether the transition is logical or not. The previous scenes were in a locksmith shop and a library, but without knowing where the characters are now, we can't evaluate the transition.
- Suggested Transition Setup: Once the current scene environment is known, a transition setup can be suggested.

Environment Detail Suggestions:
Environment Details Suggestions:
- Suggestion 1: As the characters enter a new room to open the mysterious package, the sound of an old, creaky ceiling fan spinning overhead can be heard. It's clear the room hasn't been used in a while, and one of the characters can make a joke about the fan possibly falling on them before they even get to see what's in the package.
- Suggestion 2: I

### 🧠 Scene Planner Agent


In [111]:
# Generate a plan for Scene 14 by integrating recommendations from all three agents
scene_14_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=14
)

print(scene_14_plan)

Scene Plan:
Character Goals:
- [Goal 1] Showcase Jimmy's forgetfulness and Sophie's playfulness by having Jimmy struggle to open the lock due to missing glasses, which Sophie comically replaces with sunglasses. 
- [Goal 2] Amplify Sophie's adventurous spirit and Jimmy's meticulous nature by having Sophie excitedly pull out a rubber chicken from the lock's contents, and Jimmy discovering the real treasure hidden within it.

Comedic Goal:
- [Goal] Integrate a blend of suspense and humor by having Jimmy and Sophie comment on the difficulty of the lock using a callback to a previous episode's joke, and by introducing a surprising item (socks) before revealing the true content of the lock.

Environment Detail:
- [Detail] The room where the lock is opened is dusty with an old, unreliable ceiling fan that squeaks with every rotation. An eccentrically painted ceramic giraffe stands out in the corner of the room, adding to the absurdity.

Creative Suggestion:
- [Suggestion] As the scene progres

### ✍️ Generating Scene 14 Script

In [112]:
# Generate the full script for Scene 14 based on the structured scene plan
scene_14_script = generate_scene(
    client=client,
    scene_plan=scene_14_plan,
    scene_number=14
)

print(scene_14_script)

INT. MYSTERIOUS ROOM - DAY

The room is dusty, and the ceiling fan squeaks with every rotation. An eccentrically painted ceramic giraffe stands out in the corner of the room.

Sophie, a lively young woman, and Jimmy, an older, meticulous man, are in front of a lock. Jimmy is squinting, struggling to open it.

JIMMY:
I swear I left my glasses right here.

[Laugh Track]

Sophie grins, pulling out a pair of sunglasses from her pocket.

SOPHIE:
You mean these?

[Laugh Track]

She places the sunglasses on Jimmy's face, who fumbles with the lock before it finally swings open. Sophie excitedly peeks inside.

SOPHIE:
Oh, look at this!

She pulls out a rubber chicken, squealing in delight. 

[Laugh Track]

Jimmy, less impressed, rummages through the lock and pulls out a pair of socks.

JIMMY:
Socks? Really, Sophie?

[Laugh Track]

Sophie laughs, shaking the rubber chicken. Something CLINKS inside.

SOPHIE:
Wait, there's something in the chicken!

The ceiling fan above them wobbles ominously as 

### 🗂️ Adding Metadata to Vector Database


In [113]:
# Store the generated Scene 14 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_14_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 14 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. H

## 🎬 Scene 15 Generation


In [114]:
# Extract the description for Scene 15 from the episode outline
scene_15_desc = extract_scene(episode_outline, 15)
print(scene_15_desc)

Scene 15: "Earl's Return" Earl returns to the shop, bringing his own chaos with him. The scene is chaotic and hilarious.


### 🎭 Character Agent


In [115]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 15 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_15_desc,
    scene_number=15
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [12, 13, 14]
Scene 15 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Earl's return to the shop with his own chaos is consistent with his eccentric personality trait and his ability to handle chaos, as demonstrated in previous scenes. His role in fostering teamwork and camaraderie, and his tendency to involve others in his plans, suggests that he may turn the chaos into a team-building exercise. His calm speaking style may also help alleviate the chaos.

Interaction Improvement Recommendations:
1. Earl walks into the shop with a mischievous grin and immediately starts mispronouncing idioms, causing Sophie and Jimmy to exchange knowing looks and laugh. This interaction is a nod to Scene 1, where Earl's mispronunciation of idioms is established as a family joke, and it also reinforces the warm bond between Earl, Sophie, and Jimmy.
2. Amidst the chaos, Earl decides to inv

### 🎤 Comedic Agent


In [116]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 15 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_15_desc,
    scene_number=15
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [12, 13, 14]
Scene 15 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The chaotic humor in this scene is consistent with the previous scenes, which also feature eccentric and chaotic characters and situations. The humor is derived from the characters' unconventional problem-solving methods and the unexpected situations they find themselves in.

Comedic Improvement Recommendations:
1. [Suggestion] — As Earl enters the shop, he absentmindedly knocks over a display of canned goods, causing a domino effect of cascading products. As he tries to clean up the mess, he only makes it worse, stepping on cans and slipping, causing even more commotion. This could be a funny physical comedy bit that shows Earl's chaotic nature. — (This builds on Earl's established character trait of being a bit of a clumsy oaf, which we've seen in previous episodes like "The Pie Incident" an

### 🌆 Environment Agent


In [117]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 15 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_15_desc,
    scene_number=15
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [12, 13, 14]
Scene 15 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from a Mysterious Room to a Shop is logical. The characters could have found a clue or an item in the Mysterious Room that led them to a specific Shop. This is a common narrative device used in sitcoms to move the plot forward.
- Suggested Transition Setup: Not necessary.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As soon as the characters enter the shop, the jingling of a small bell attached to the door signals their arrival. Earl, the shop owner, is a quirky character who's seen squinting through a magnifying glass at a seemingly ordinary item - a toaster. He jumps at the sound of the bell, dropping the toaster, which then surprisingly pops out perfectly toasted bread.
- [Suggestion 2]: The shop is filled with the mixed scent o

### 🧠 Scene Planner Agent


In [118]:
# Generate a plan for Scene 15 by integrating recommendations from all three agents
scene_15_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=15
)

print(scene_15_plan)

Scene Plan:
Character Goals:
- [Goal 1] Showcase Earl's eccentric personality and the camaraderie he shares with Sophie and Jimmy through a playful interaction built on their inside family joke about his idiom mispronunciations.
- [Goal 2] Further establish Earl's leadership and his knack for involving his coworkers in unconventional plans to restore order amidst chaos.

Comedic Goal:
- [Goal] Create a humorous sequence by escalating the chaos in the shop, initiated by Earl's clumsy mishap, and furthered by Tina's well-intentioned but embarrassingly loud attempts to control the situation.

Environment Detail:
- [Detail] The shop, an embodiment of Earl's quirkiness, is filled with an unusual blend of scents and sounds - the faint jingle of a bell that announces every customer's arrival, a whiff of Earl's strong cologne, the nostalgic smell of old books and a lingering smell of burnt toast.

Creative Suggestion:
- [Suggestion] As the chaos continues to spiral, Earl, in an attempt to rest

### ✍️ Generating Scene 15 Script

In [119]:
# Generate the full script for Scene 15 based on the structured scene plan
scene_15_script = generate_scene(
    client=client,
    scene_plan=scene_15_plan,
    scene_number=15
)

print(scene_15_script)

INT. EARL'S SHOP - DAY

Earl, Sophie, and Jimmy are gathered around the counter, chuckling. Earl holds up a vintage toy helicopter.

EARL:
You see this? This is our ticket to restoring order in here.

[Sophie and Jimmy exchange a confused look. Laugh Track]

SOPHIE:
Earl, you're not going to start saying your idioms wrong again, are you?

EARL:
(Laughs)
You know me too well, Soph. Let's just say, we're about to... kill two birds with one chopper!

[Laugh Track]

Suddenly, the door swings open. The bell jingles. Enter Tina, tripping over a pile of books. 

TINA:
(Shrieks)
Oh my goodness, what a mess!

[Laugh Track]

Earl, unfazed by the chaos, looks at Tina and winks.

EARL:
Tina, just in time. Could use your voice to guide this little bird.

Sophie snickers. Jimmy shakes his head, amused.

JIMMY:
(Laughs)
Earl, you do realize that's not how the saying goes, right?

[Laugh Track]

Earl grins, knowingly.

EARL:
Oh, I know. It’s just more fun this way. Now, are you in, or are you out?

So

### 🗂️ Adding Metadata to Vector Database


In [120]:
# Store the generated Scene 15 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_15_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 15 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. H

## 🎬 Scene 16 Generation


In [121]:
# Extract the description for Scene 16 from the episode outline
scene_16_desc = extract_scene(episode_outline, 16)
print(scene_16_desc)

Scene 16: "Shared Histories" Jimmy, Sophie, and Earl share stories from their past, adding depth to their relationships. The tone is humorous and sentimental.


### 🎭 Character Agent


In [122]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 16 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_16_desc,
    scene_number=16
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [13, 14, 15]
Scene 16 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: The planned scene aligns well with the established character profiles. Jimmy's meticulous nature and role as the voice of reason, Sophie's adventurous and resilient spirit, and Earl's eccentric problem-solving approach all make sense in a scenario where they share past stories. Their individual speaking styles and quirks, along with their relationships with each other, can be highlighted through this storytelling. Their actions and dialogue can logically stem from their past interactions. No contradictions are apparent based on past scenes.

Interaction Improvement Recommendations:
1. Jimmy finally finds his glasses and Sophie teases him by saying, "Are those your lock picks or are you just happy to see me?" — This interaction builds upon the running joke from Scene 1 where Jimmy mistakenly used his 

### 🎤 Comedic Agent


In [123]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 16 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_16_desc,
    scene_number=16
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [13, 14, 15]
Scene 16 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The scene maintains the humorous tone of the previous scenes while adding a layer of sentimentality. The characters' shared histories and relationships are explored, adding depth to their interactions while keeping the humor intact.

Comedic Improvement Recommendations:
1. [Suggestion] — Have Jimmy recount a story from his past where he joined a school talent show as a magician, intending to impress his crush. Unfortunately, the trick goes horribly wrong, and he ends up accidentally releasing a live dove in the auditorium, causing pandemonium. This story can be introduced when Sophie shares her own embarrassing school story, creating a bonding moment between them. This also adds to Jimmy's character, showing his willingness to take risks for people he cares about, even if it leads to embarrass

### 🌆 Environment Agent


In [124]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 16 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_16_desc,
    scene_number=16
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [13, 14, 15]
Scene 16 — Environment Transition: ❌ Inconsistent

Explanation:
Short Explanation: The current scene environment is not specified, making it impossible to determine if the transition is logical or not. The audience needs to know where the characters are to understand the context of the scene.
- Suggested Transition Setup: Depending on the plot, the characters could leave Earl's Shop and head to a new location. For example, if they found a clue in Earl's Shop, they could be seen heading to the location the clue leads to, such as a local café or a character's home.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As the characters walk into the local café, the sound of an old, off-key jukebox fills the room, playing an ironically cheerful song that contrasts with their serious investigation. A barista, in the background, is struggl

### 🧠 Scene Planner Agent


In [125]:
# Generate a plan for Scene 16 by integrating recommendations from all three agents
scene_16_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=16
)

print(scene_16_plan)

Scene Plan:
Character Goals:
- [Goal 1] Jimmy finds his glasses, and Sophie engages in playful banter with him, referencing the recurring joke of Jimmy's glasses being used as lock picks. This interaction should solidify their established dynamic and heighten their camaraderie.
- [Goal 2] Earl proposes a nostalgic trip to their old locksmith shop, prompting Sophie's humorous response. This should emphasize Sophie's adventurous spirit and her playful tolerance of Earl's quirky problem-solving methods.

Comedic Goal:
- [Goal] Jimmy shares a humorous and embarrassing story from his past, demonstrating his endearing clumsiness and willingness to take risks for love. Concurrently, Earl interjects with incorrect details from shared experiences, reinforcing his whimsical character and creating a running gag.

Environment Detail:
- [Detail] As they enter the locksmith shop, the familiar, metallic scent of iron keys and the sound of rusty lock mechanisms fill the air. In the background, a cucko

### ✍️ Generating Scene 16 Script

In [126]:
# Generate the full script for Scene 16 based on the structured scene plan
scene_16_script = generate_scene(
    client=client,
    scene_plan=scene_16_plan,
    scene_number=16
)

print(scene_16_script)

INT. LOCKSMITH SHOP - DAY

The door CREAKS open. SOPHIE, EARL, and JIMMY step inside. The metallic scent of iron keys and the SOUND of rusty lock mechanisms fill the air.

SOPHIE: (sniffs) Ah, the sweet smell of iron and rust. Takes me back.

EARL: (nodding) Yep. Just like old times.

JIMMY is searching frantically in his pockets.

SOPHIE: (smirking) Looking for your glasses, Jimmy?

JIMMY: (nodding) Yeah, I can't seem to find them.

SOPHIE: (grinning) Well, if we ever need to pick a lock, we know where to look.

[Laugh Track]

EARL: (laughing) Oh, that's a good one, Sophie!

JIMMY: (smiling) Very funny, Sophie. 

Jimmy finds his glasses and puts them on.

JIMMY: (looking around) This place hasn't changed a bit, has it?

EARL: (excited) You know what this place needs? A trip down memory lane!

SOPHIE: (teasing) What, like a séance to summon the spirit of the locksmith we once knew?

[Laugh Track]

EARL: (laughs) No, no. But remember when Jimmy tried to impress Cindy at the school talen

### 🗂️ Adding Metadata to Vector Database


In [127]:
# Store the generated Scene 16 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_16_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 16 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. H

## 🎬 Scene 17 Generation


In [128]:
# Extract the description for Scene 17 from the episode outline
scene_17_desc = extract_scene(episode_outline, 17)
print(scene_17_desc)

Scene 17: "Locking Up" As they close the shop, Jimmy, Sophie, and Earl reflect on the day's events. The atmosphere is light and reflective.


### 🎭 Character Agent


In [129]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 17 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_17_desc,
    scene_number=17
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [14, 15, 16]
Scene 17 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: The planned scene aligns with the character profiles of Jimmy, Sophie, and Earl. All three have shown a tendency to reflect on past events, shown by Jimmy's and Earl's nostalgia and Sophie's reminiscing about future adventures. Their light-hearted camaraderie in past scenes also suggests they would approach this reflective situation with humor and warmth, consistent with their established personalities.

Interaction Improvement Recommendations:
1. Sophie teases Jimmy about his meticulousness, suggesting they could have closed up faster if he wasn't so fixated on details like arranging the keys in perfect order. She could end it with a lighthearted comment, "We might be here till Christmas at this rate, Jimmy!" — (This interaction reflects Sophie's lively nature and her playful relationship with Jimmy

### 🎤 Comedic Agent


In [130]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 17 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_17_desc,
    scene_number=17
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [14, 15, 16]
Scene 17 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The tone of reflective humor is consistent with the previous scenes. The characters are reflecting on the day's events, which is a common theme in the sitcom. The humor is light and consistent with the characters' personalities and the overall comedic tone of the show.

Comedic Improvement Recommendations:
1. [Suggestion] — As Jimmy, Sophie, and Earl are cleaning up, Jimmy struggles to balance a stack of boxes. Sophie offers to help, but he insists he's got it under control. The boxes inevitably topple over, sending merchandise rolling across the floor. The punchline: "Well, I guess I'm not cut out for juggling after all!" — This adds physical humor and references Jimmy's earlier failed attempt to learn juggling during their lunch break.
2. [Suggestion] — As they're turning off the lights and 

### 🌆 Environment Agent


In [131]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 17 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_17_desc,
    scene_number=17
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [14, 15, 16]
Scene 17 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The previous scenes were all set in different types of shops or rooms that could be associated with shops. Therefore, transitioning to another shop environment is logical and believable within the context of the show.
- Suggested Transition Setup: Not necessary.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As the characters enter the shop, a small, quirky wind chime made of kitchen utensils hanging by the door jingles melodically. This could be a funny visual element and also add a unique sound cue to the scene.
- [Suggestion 2]: The shop owner is seen in the background trying to manage a misbehaving parrot, which keeps trying to steal customers' purchases. This can add a humorous element to the scene, and also provide a constant background actio

### 🧠 Scene Planner Agent


In [132]:
# Generate a plan for Scene 17 by integrating recommendations from all three agents
scene_17_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=17
)

print(scene_17_plan)

Scene Plan:
Character Goals:
- [Goal 1] Establish a playful dynamic between Sophie and Jimmy by having Sophie tease Jimmy about his obsessive attention to detail. This should be done in a way that reflects Sophie's lively nature and Jimmy's meticulous character.
- [Goal 2] Highlight Earl's nostalgic side and the group's cooperative spirit by having Earl suggest a joint project of restoring an old, broken lock.

Comedic Goal:
- [Goal] Introduce physical humor and play on established character traits by having Jimmy struggle with a stack of boxes and Earl losing track of keys.

Environment Detail:
- [Detail] The locksmith shop should have a unique, quirky charm. This could be shown through a whimsical wind chime made from locksmith tools and a mischievous pet in the background causing light-hearted chaos.

Creative Suggestion:
- [Suggestion] The scene could naturally progress with the group deciding to stay back after closing hours to work on restoring the old lock together. This would b

### ✍️ Generating Scene 17 Script

In [133]:
# Generate the full script for Scene 17 based on the structured scene plan
scene_17_script = generate_scene(
    client=client,
    scene_plan=scene_17_plan,
    scene_number=17
)

print(scene_17_script)

INT. LOCKSMITH SHOP - DAY

Sophie, Jimmy, and Earl are gathered around the counter. A whimsical wind chime made from locksmith tools hangs above them, clinking gently. A mischievous parrot squawks from its cage in the background.

SOPHIE, holding a broken old lock:
You know, Jimmy, I bet you could fix this thing with your eyes closed.

[Laugh Track]

JIMMY, blushing:
Well, I do know my way around a lock.

SOPHIE, teasing:
Oh, you know your way around everything! You'd probably alphabetize a bowl of alphabet soup.

[Laugh Track]

Jimmy looks embarrassed but laughs, balancing a precarious stack of boxes. Suddenly, the top box slides off, causing Jimmy to stumble. The parrot squawks loudly in the background.

[Laugh Track]

EARL, chuckling:
Now, there's an idea. We should restore this old lock together. Brings back memories of my apprenticeship.

Sophie and Jimmy exchange glances, then nod.

SOPHIE, excited:
That sounds like a great idea, Earl. Let's do it!

Earl pulls out a dusty box fil

### 🗂️ Adding Metadata to Vector Database


In [134]:
# Store the generated Scene 17 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_17_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 17 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. H

## 🎬 Scene 18 Generation


In [135]:
# Extract the description for Scene 18 from the episode outline
scene_18_desc = extract_scene(episode_outline, 18)
print(scene_18_desc)

Scene 18: "Earl's Surprise" Earl reveals his personal issue has been resolved in an unexpected way. The scene is surprising and comical.


### 🎭 Character Agent


In [136]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 18 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_18_desc,
    scene_number=18
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [15, 16, 17]
Scene 18 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Earl's personality is consistent with his established traits. His resolution of a personal issue in an unexpected way aligns with his problem-solving trait, as well as his eccentricity. The comedic nature of the scene is in line with his jovial personality. His dialogue and actions, if centered around his idiosyncratic use of idioms and nostalgic reminiscences, would also be consistent. Without specifics, it's hard to determine whether there are any contradictions or missing motivations.

Interaction Improvement Recommendations:
1. Earl, in his typical eccentric fashion, uses a toy helicopter to drop an envelope from the air revealing that his personal issue has been resolved. The group laughs and congratulates him, Sophie teases him about his dramatic reveal — (This interaction is in line with Earl'

### 🎤 Comedic Agent


In [137]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 18 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_18_desc,
    scene_number=18
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [15, 16, 17]
Scene 18 — Comedic Tone: ❌ Inconsistent

Comedic Tone Analysis:
3. Short Explanation: The scene description suggests a continuation of the comedic and surprising tone that has been consistent throughout the previous scenes. The characters' camaraderie and humor are expected to continue in this scene.

Comedic Improvement Recommendations:
1. [Suggestion] — Have Earl initially dodge the question about how his personal issue was resolved. He could use humor and diversion tactics to avoid answering, such as saying, "Well, you know, it's a funny story. It all started with a banana peel and a pair of roller skates..." Then, when pressed, he reveals the surprising resolution in a nonchalant manner. This builds on Earl's character trait of using humor to deflect from personal matters, established in prior scenes. 
2. [Suggestion] — Introduce a light running joke about Earl's outrageo

### 🌆 Environment Agent


In [138]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 18 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_18_desc,
    scene_number=18
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [15, 16, 17]
Scene 18 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from the locksmith shop to the local police station is logical. It could be that the characters needed to report a theft or a break-in, which is why they were at the locksmith's in the first place. The sequence of events makes sense in the context of the show. 
- Suggested Transition Setup: Not necessary.

Environment Detail Suggestions:
Environment Details Suggestions:
- As the characters enter the police station, a cop walks by with a donut in one hand and a coffee in the other, accidentally bumping into one of the characters and spilling a bit of coffee on them.
- The faint sound of a police radio can be heard in the background, occasionally interrupted by the loud ringing of the station's landline phone.

Environment Agent's Internal Thoughts:
- Think: Retrieved sum

### 🧠 Scene Planner Agent


In [139]:
# Generate a plan for Scene 18 by integrating recommendations from all three agents
scene_18_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=18
)

print(scene_18_plan)

Scene Plan:
Character Goals:
- [Goal 1] Earl should creatively reveal his personal issue resolution using an unexpected prop (a toy helicopter), demonstrating his quirkiness and problem-solving nature.
- [Goal 2] During the group's celebration, Earl should misuse an idiom, leading to a humorous correction from Tina and a nostalgic, sidetracking story from Earl that underscores the warm, familial dynamics between the characters.

Comedic Goal:
- [Goal] Earl should initially dodge questions about how he resolved his personal issue using humor and diversion, then nonchalantly reveal the surprising solution. Integrate a running joke about Earl's unorthodox problem-solving methods, drawing disbelief and humor from other characters.

Environment Detail:
- [Detail] In the police station, an officer is seen struggling to manage a stack of paperwork, adding to the bustling ambiance. The papers topple over, causing a moment of slapstick comedy.

Creative Suggestion:
- [Suggestion] As the group e

### ✍️ Generating Scene 18 Script

In [140]:
# Generate the full script for Scene 18 based on the structured scene plan
scene_18_script = generate_scene(
    client=client,
    scene_plan=scene_18_plan,
    scene_number=18
)

print(scene_18_script)

INT. POLICE STATION - DAY

[The group enters the busy station. A police OFFICER struggles with a stack of paperwork, which topples over.]

[Laugh Track]

EARL, distracted, stumbles in holding a toy helicopter.

TINA: Earl, what's with the toy helicopter?

EARL: Oh, this little thing? Just a...uh...a stress reliever. 

[Laugh Track]

TINA: And how exactly does that relieve your stress?

EARL: Well, it's a long story...

TINA: More like a tall tale.

[Laugh Track]

EARL: Okay, okay. It actually helped me solve a problem.

TINA: Really? How?

EARL: Well, I tied my unpaid parking tickets to it and flew it straight into the payment window.

[Laugh Track]

TINA: You're kidding, right?

EARL: Not at all. You know I have a knack for unconventional problem-solving.

TINA: That's one way to put it...

[Laugh Track]

OFFICER: (on phone) Yes, that's correct. A toy helicopter delivered the payment.

[Group laughs]

EARL: I guess you could say, I killed two stones with one bird.

TINA: (laughs) Earl

### 🗂️ Adding Metadata to Vector Database


In [141]:
# Store the generated Scene 18 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_18_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 18 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. H

## 🎬 Scene 19 Generation


In [142]:
# Extract the description for Scene 19 from the episode outline
scene_19_desc = extract_scene(episode_outline, 19)
print(scene_19_desc)

Scene 19: "A New Beginning" Sophie decides to stick around and learn more about locksmithing, signifying a new start for her and Jimmy. The tone is hopeful and heartwarming.


### 🎭 Character Agent


In [143]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 19 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_19_desc,
    scene_number=19
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [16, 17, 18]
Scene 19 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Sophie's decision to stick around and learn about locksmithing aligns with her cooperative and participative nature from previous scenes. Her playful interaction with Jimmy also remains consistent. Jimmy's willingness to teach Sophie also aligns with his cooperative and good-natured personality. The hopeful tone of this scene is in line with their shared optimism about the shop's future from previous scenes.

Interaction Improvement Recommendations:
1. Sophie enthusiastically proposes a detailed plan for reorganizing the shop, showcasing a growth in her leadership skills. Jimmy, in his usual meticulous fashion, suggests tweaks to improve her plan. — This interaction reflects the cooperative and engaged nature of both characters, as seen in all three previous scenes. Moreover, it builds upon Sophie's 

### 🎤 Comedic Agent


In [144]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 19 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_19_desc,
    scene_number=19
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [16, 17, 18]
Scene 19 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The heartwarming humor in the new scene is consistent with the friendly banter and camaraderie in the previous scenes. The humor is derived from the characters' interactions and their shared history, which has been a consistent theme.

Comedic Improvement Recommendations:
1. [Suggestion] — As Sophie decides to stick around, she could attempt to showcase her novice locksmithing skills by trying to pick a lock on an old safe in the shop. However, instead of succeeding, she accidentally locks it even tighter. This could lead to a funny exchange where Jimmy, trying not to discourage her, says, "Well, you've certainly got a knack for making sure things stay locked!" — (This adds humor while highlighting Sophie's inexperience and Jimmy's supportive nature. It also provides a visual, physical comedy 

### 🌆 Environment Agent


In [145]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 19 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_19_desc,
    scene_number=19
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [16, 17, 18]
Scene 19 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from the police station back to the locksmith shop is logical. It's plausible that after an incident that required police involvement, the characters would return to the locksmith shop, which is presumably their usual setting or place of work.
- Suggested Transition Setup: Not necessary.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As the characters enter the locksmith shop, the jingle of a small bell attached to the door can be heard, indicating customers coming in. This could be a running gag in the series where the bell jingles at inappropriate or comedic moments.
- [Suggestion 2]: A giant key prop is seen in the background, hanging on the wall as a quirky and funny decoration. It adds a touch of humor and personality to the sho

### 🧠 Scene Planner Agent


In [146]:
# Generate a plan for Scene 19 by integrating recommendations from all three agents
scene_19_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=19
)

print(scene_19_plan)

Scene Plan:

Character Goals:
- [Goal 1]: Showcase Sophie's growing confidence and leadership skills as she enthusiastically proposes a detailed plan to reorganize the shop, while also displaying her playful side as she teases Jimmy about a past mistake.
- [Goal 2]: Highlight Jimmy's meticulous nature and his supportive attitude towards Sophie by having him suggest improvements to Sophie's plan, reacting positively to her playful teasing, and encouraging her when she makes a mistake.

Comedic Goal:
- [Goal]: Introduce a funny exchange where Sophie, in her enthusiasm, misnames various types of locks and keys with fictional terms from sci-fi movies, leading to Jimmy gently correcting her in a humorous manner.

Environment Detail:
- [Detail]: The locksmith shop is cluttered with various types of locks, keys, and locksmithing tools, adding to the quirky atmosphere. A comically oversized padlock could be displayed on the counter, providing a potential prop for future comedic scenes.

Creati

### ✍️ Generating Scene 19 Script

In [147]:
# Generate the full script for Scene 19 based on the structured scene plan
scene_19_script = generate_scene(
    client=client,
    scene_plan=scene_19_plan,
    scene_number=19
)

print(scene_19_script)

INT. LOCKSMITH SHOP - DAY

[Sophie is behind the counter, surrounded by a clutter of keys, locks, and locksmithing tools. She is animatedly gesturing at a makeshift blueprint of the shop.]

SOPHIE:
...and then we move the key duplicator to the back! Not like last time when Jimmy put it next to the furnace and nearly melted the whole batch of keys.

[Sophie winks at Jimmy, who is leaning against the counter, arms folded. Laugh Track.]

JIMMY:
Well, you live and you learn. I like your plan, Sophie, but what about the cabinet of antique locks? It's pretty heavy...

SOPHIE:
Oh, we'll just use the quantum levitator to move it!

[Jimmy raises an eyebrow. Laugh Track.]

JIMMY:
Quantum levitator? You've been watching too many sci-fi movies again, haven't you?

[Sophie grins sheepishly.]

SOPHIE:
Okay, okay. Maybe I got a bit carried away. But we can use the dolly, right?

JIMMY:
That's more like it. And remember, it's better to make mistakes now than when we're reorganizing.

[Sophie nods, loo

### 🗂️ Adding Metadata to Vector Database


In [148]:
# Store the generated Scene 19 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_19_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 19 


Scene 1
Summary: In this opening scene, Jimmy, a locksmith in his mid-40s, starts his day at his shop. He begins with a quirky morning routine of trying to pick a lock with a hairpin. His cat, Whiskers, surprises him by jumping on the counter, leading to a humorous interaction involving a master key and a can of tuna. A gust of wind causes a stack of keys to fall, and Jimmy takes a tumble trying to catch them. Despite the mishap, he dusts himself off and prepares to face the day's challenges, showing his resilience and good humor.
Characters: ['Jimmy', 'Whiskers']
Location: Jimmy's Locksmith Shop
Recurring Joke: Jimmy's attempts to use keys for non-lock related tasks (e.g., using the master key to open a can of tuna) and his playful banter with his cat, Whiskers, are recurring jokes.
Emotional Tone: Humorous, Light-hearted

Scene 2
Summary: In the scene, Jimmy, a man with unconventional habits, tries to open a mysterious package with a hairpin. H

## 🎬 Scene 20 Generation


In [149]:
# Extract the description for Scene 20 from the episode outline
scene_20_desc = extract_scene(episode_outline, 20)
print(scene_20_desc)

Scene 20: "Closing the Day" The day ends with Jimmy, Sophie, and Earl sharing a laugh, promising more hilarious adventures to come. The atmosphere is light and funny.


### 🎭 Character Agent


In [150]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 20 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_20_desc,
    scene_number=20
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [17, 18, 19]
Scene 20 — Consistency: ✅ Consistent

Explanation:
1. Consistency Verdict (Yes/No)
   Yes
2. Short Explanation Why (max 5 lines)
   Each character's behavior aligns well with their established traits and arcs. Jimmy's good-natured humor, Sophie's resourcefulness and playful teasing, and Earl's nostalgic reminiscing and unconventional problem-solving are all consistent with their earlier scenes. The light and funny atmosphere of the closing scene fits their shared camaraderie and ongoing banter.

Interaction Improvement Recommendations:
1. Sophie teases Jimmy by placing his glasses near a lock and saying, "In case you need to pick this one too, Jimmy." Then she chuckles and hands him a real lockpick set. — This interaction fits because it brings back the running joke about Jimmy's glasses being used to pick locks from Scene 1 and also highlights Sophie's playful nature as seen

### 🎤 Comedic Agent


In [151]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 20 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_20_desc,
    scene_number=20
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [17, 18, 19]
Scene 20 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The tone of the new scene is consistent with the previous scenes. The characters continue to engage in friendly banter and playful humor, which has been a consistent theme throughout the sitcom.

Comedic Improvement Recommendations:
1. [Suggestion] — As the day ends, Jimmy, the klutz of the group, attempts to turn off the lights to set the mood for their next adventure planning. However, he accidentally hits the disco ball switch instead, causing a mini dance party to erupt in the room. Jimmy, flustered, fumbles to turn off the disco ball and in his panic, he stumbles over a chair, causing Sophie and Earl to burst out laughing. — (This builds on Jimmy's established clumsiness from previous scenes and adds an unexpected, yet humorous element to their end-of-the-day gathering. It also sets the s

### 🌆 Environment Agent


In [152]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 20 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_20_desc,
    scene_number=20
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [17, 18, 19]
Scene 20 — Environment Transition: ❌ Inconsistent

Explanation:
Short Explanation: The current scene environment is unspecified, which makes it impossible to determine whether the transition is logical or not. The previous scenes were in a locksmith shop and a police station, so the next scene should logically follow from these locations.
- Suggested Transition Setup: The main character could receive a phone call while at the locksmith shop, asking them to meet at a new location. This would provide a logical transition to a new scene.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: The main character arrives at the new location - a crowded, hipster coffee shop. The barista is struggling to spell a ridiculously long name on a coffee cup, while another customer loudly debates the merits of different types of non-dairy milk. 
- [Su

### 🧠 Scene Planner Agent


In [153]:
# Generate a plan for Scene 20 by integrating recommendations from all three agents
scene_20_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=20
)

print(scene_20_plan)

Scene Plan:
Character Goals:
- [Goal 1]: To enhance the playful dynamic between Sophie and Jimmy, Sophie should make a humorous remark about Jimmy's lock-picking abilities, subtly referencing their past adventures. This can be achieved by having Sophie give Jimmy an actual lockpick set, hinting at the running joke about his glasses.
- [Goal 2]: Earl should continue to play the role of the light-hearted jester in the group by making a playful comment about Jimmy's past blunders, specifically his knack for locking himself in trunks. This will serve to highlight his character's jovial nature and also continue a running joke from previous scenes.

Comedic Goal:
- [Goal]: The scene should include a comedic incident where Jimmy's clumsiness is showcased. This could be Jimmy attempting to create a serious atmosphere for their adventure planning but inadvertently causing a hilarious situation. This comedic moment should be followed by a light-hearted group banter that highlights their bond and

### ✍️ Generating Scene 20 Script

In [154]:
# Generate the full script for Scene 20 based on the structured scene plan
scene_20_script = generate_scene(
    client=client,
    scene_plan=scene_20_plan,
    scene_number=20
)

print(scene_20_script)

INT. COFFEE SHOP - DAY

Sophie, Jimmy, and Earl are huddled around a small table. The barista struggles with a long coffee order. A CUSTOMER loudly complains about the lack of non-dairy milk. Another patron obliviously slurps soup.

SOPHIE
(to Jimmy)
Here, I thought you might need this.

Sophie hands Jimmy a lockpick set.

JIMMY
(confused)
What's this for?

SOPHIE
(smiling)
Just in case you need to pick your way out of a trunk... again.

[Laugh Track]

EARL
(laughs)
Or your glasses.

[Laugh Track]

Jimmy, slightly embarrassed but laughing, attempts to regain control of the situation.

JIMMY
(serious)
Alright, enough with the jokes. We need to plan this adventure.

As Jimmy stands up to dim the lights, he accidentally hits a switch, starting a disco ball. The room is filled with rotating, colorful lights, and disco music starts playing from the coffee shop speakers.

[Laugh Track]

Sophie and Earl burst into laughter. The barista and other customers look at Jimmy, surprised but also amu

## Combining Scripts



In [155]:
# List your scene scripts in order
scene_scripts = [
    scene_1_script,
    scene_2_script,
    scene_3_script,
    scene_4_script,
    scene_5_script,
    scene_6_script,
    scene_7_script,
    scene_8_script,
    scene_9_script,
    scene_10_script,
    scene_11_script,
    scene_12_script,
    scene_13_script,
    scene_14_script,
    scene_15_script,
    scene_16_script,
    scene_17_script,
    scene_18_script,
    scene_19_script,
    scene_20_script
]

# Combine with optional scene headers
full_episode_script = "\n\n".join([
    f"### Scene {i+1} ###\n{script.strip()}"
    for i, script in enumerate(scene_scripts)
])

# Preview the first 500 characters
print(full_episode_script[:500])


### Scene 1 ###
# Scene 1

INT. JIMMY'S LOCKSMITH SHOP - MORNING

JIMMY, mid 40s, opens the door to his shop, a jumble of keys jingling from his belt. He flips the sign from CLOSED to OPEN. He's wearing a t-shirt that says, "I've Got the Key to Your Heart."

JIMMY
(To himself)
Another day, another lock to pick. 

Jimmy begins his peculiar morning routine. He pulls out a lock from his pocket and tries to pick it using a hairpin.

JIMMY
(To the lock)
You're a tough one, aren't you? 

[LAUGH TRACK]


NameError: name 'stop' is not defined

In [157]:
path = "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/pilot episode data/pilot episode 3/react_pilot_ep_scrpt_3.txt"

with open(path, "w") as f:
    f.write(full_episode_script)

print(f"Script saved to {path}")

Script saved to /content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/pilot episode data/pilot episode 3/react_pilot_ep_scrpt_3.txt
